In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
# Set pandas display options to show up to 100 rows
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', None)

In [3]:
data = pd.read_csv('data/lower/ALLSLDL_DP05_AL.csv', dtype={'Line Number': str})

In [4]:
# Load in Alabama DEMOGRAPHIC file for all districts

data.rename(columns={
    'GEONAME': 'District',
    'Line Number': 'PROFLN',
    'Title':'TITLE',
    'Estimate': 'PRF_ESTIMATE',
    'Percent Estimate':'PCT_ESTIMATE', 
    'Margin of Error': 'PRF_MG_ERROR',
    'Percent Margin of Error': 'PCT_MG_ERROR'
}, inplace=True)

# 'Title':'TITLE',
#     'TITLE': 'Title',
# make a dictionary of Line Number keys to Title values
title_dict = dict(zip(data['PROFLN'], data['TITLE']))

# convert the dictionary to dataframe and observe 
title_df = pd.DataFrame(list(title_dict.items()), columns=['PROFLN', 'TITLE'])
title_df.head(10)

,PROFLN,TITLE
0,0,ACS DEMOGRAPHIC AND HOUSING ESTIMATES
1,0.5,NaN
2,0.8,SEX AND AGE
3,1,Total population
4,2,Male
5,3,Female
6,4,Sex ratio (males per 100 females)
7,4.3,NaN
8,5,Under 5 years
9,6,5 to 9 years


In [5]:
## Combining PROFLN numbers which will become new TITLEs

# "Total Population-" Category
for i in range(2, 25):
    data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '1-' + str(i)

# "Total Population- 18 years and over-" Category
for i in range(25, 29):
    data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '1-25' + ('' if i == 25 else '-' + str(i))

#"Total Population- 65 years and over-" Category
for i in range(29, 33):
    data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '1-29' + ('' if i == 29 else '-' + str(i))

# "RACE-" Category
for i in range(33, 37):
    data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '32.5-' + str(i)

# "RACE- One race-" Category
for i in range(37, 58):
    data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '32.5-36-' + str(i)

# "RACE- Two or more races-" Category
for i in range(58, 63):
    data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '32.5-58' + ('' if i == 58 else '-' + str(i))

#"RACE- Race alone or in combination with one or more other races-" Category
for i in range(63, 70):
    data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '32.5-62.5-' + str(i)

# 'HISPANIC OR LATINO AND RACE-" Category
for i in range(70, 86):
    data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '69.5-' + str(i)

# 'CITIZEN, VOTING AGE POPULATION' Category
for i in range(87, 90):
    data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '86.5-' + str(i)

In [6]:
# Translate new profln numbers to combo titles
def translate_profln(profln):
    # split the PROFLN value into individual keys
    keys = profln.split('-')

    # translate each key from the title_dict
    titles = [str(title_dict.get(key, '')) for key in keys]

    # concat translated title strings
    translated_title = ' - '.join(title for title in titles if title)

    return translated_title

# replace TITLE values with their new names using function
data['TITLE'] = data['PROFLN'].apply(translate_profln)
data['TITLE'].value_counts()


TITLE
nan                                                                                                              1260
RACE - Two or more races                                                                                          210
RACE - One race                                                                                                   210
Total population - 65 years and over                                                                              210
Total population - 18 years and over                                                                              210
RACE - One race - Native Hawaiian                                                                                 105
RACE - Two or more races - Black or African American and American Indian and Alaska Native                        105
RACE - One race - Other Asian                                                                                     105
RACE - Race alone or in combination with one or mo

In [7]:
# remove rows with NaN's, placeholder 'N' for 'Not Large Enough Sample Size', and placeholder '(X)'' rows
invalid_rows = data['PRF_ESTIMATE'].isin(['N', '(X)']) | data['PRF_ESTIMATE'].isna()

# filter out these rows from the Dataframe
data = data[~invalid_rows]


In [8]:
# remove rows of duplicate information or strong overlap

# List of PROFLN values to remove (many values are repeats from other categories and subcategories)
profln_to_remove = ['1-29', '1-25', '32.5-33', '32.5-36', '32.5-58', 
                    '32.5-62.5-63', '32.5-62.5-64', '32.5-62.5-65', 
                    '32.5-62.5-66', '32.5-62.5-67', '32.5-62.5-68', 
                    '32.5-62.5-69', '69.5-70', '69.5-76', '69.5-77', 
                    '69.5-78', '69.5-79', '69.5-80', '69.5-81', '69.5-82']

# filter the data to observe which rows we are removing
rows_to_remove = data[data['PROFLN'].isin(profln_to_remove)]
print(rows_to_remove['TITLE'])

# Remove these rows from the DataFrame
data = data[~data['PROFLN'].isin(profln_to_remove)]

31                                                 Total population - 18 years and over
36                                                 Total population - 65 years and over
42                                                              RACE - Total population
46                                                                      RACE - One race
68                                                             RACE - Two or more races
                                              ...                                      
11220                     HISPANIC OR LATINO AND RACE - Black or African American alone
11221             HISPANIC OR LATINO AND RACE - American Indian and Alaska Native alone
11222                                         HISPANIC OR LATINO AND RACE - Asian alone
11223    HISPANIC OR LATINO AND RACE - Native Hawaiian and Other Pacific Islander alone
11224                               HISPANIC OR LATINO AND RACE - Some other race alone
Name: TITLE, Length: 2100, dtype

In [9]:
# Delete margin of error columns
data = data.drop(columns=['PRF_MG_ERROR', 'PCT_MG_ERROR'])

# Some MEDIAN and MEAN values are not listed in PCT_ESTIMATE column, but are relevant
data.loc[data['PCT_ESTIMATE'] == '(X)', 'PCT_ESTIMATE'] = data['PRF_ESTIMATE']

# Convert all the values to numeric
data['PCT_ESTIMATE'] = data['PCT_ESTIMATE'].str.replace(',', '').astype(float)
data['PRF_ESTIMATE'] = data['PRF_ESTIMATE'].str.replace(',', '').astype(float)

data

,GEOID,District,PROFLN,TITLE,PRF_ESTIMATE,PCT_ESTIMATE
3,620L800US01001,"State House District 1 (2022), Alabama",1,Total population,48497.0,48497.0
4,620L800US01001,"State House District 1 (2022), Alabama",1-2,Total population - Male,23332.0,48.1
5,620L800US01001,"State House District 1 (2022), Alabama",1-3,Total population - Female,25165.0,51.9
6,620L800US01001,"State House District 1 (2022), Alabama",1-4,Total population - Sex ratio (males per 100 females),92.7,92.7
8,620L800US01001,"State House District 1 (2022), Alabama",1-5,Total population - Under 5 years,2386.0,4.9
...,...,...,...,...,...,...
11227,620L800US01105,"State House District 105 (2022), Alabama",69.5-85,"HISPANIC OR LATINO AND RACE - Two races excluding Some other race, and Three or more races",943.0,2.0
11229,620L800US01105,"State House District 105 (2022), Alabama",86,Total housing units,21053.0,21053.0
11232,620L800US01105,"State House District 105 (2022), Alabama",86.5-87,"CITIZEN, VOTING AGE POPULATION - Citizen, 18 and over population",35300.0,35300.0
11233,620L800US01105,"State House District 105 (2022), Alabama",86.5-88,"CITIZEN, VOTING AGE POPULATION - Male",16805.0,47.6


In [10]:
## Converting the long district titles to their state and district codes ('AL-01' = 'Alabama District 1')
# State Mapping
states = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA',
    'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA',
    'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA',
    'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD',
    'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
    'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH',
    'New Jersey': 'NJ', 'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC',
    'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK', 'Oregon': 'OR', 'Pennsylvania': 'PA',
    'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD', 'Tennessee': 'TN',
    'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA',
    'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'
}

def format_house_district(district):
    # Regular expression to extract the district number and state from the new format
    match = re.search(r'State House District (\d+) \(\d+\), ([A-Za-z\s]+)', district)
    if match:
        district_num, state_name = match.groups()
        district_num = district_num.zfill(2)  # Pad the district number with leading zeros if necessary
        state_abbr = states.get(state_name.strip(), state_name)  # Get the state abbreviation
        return f'{state_abbr}-House-{district_num}'  # Format it as required
    return district  # Return the original if no match found


data['Formatted_District'] = data['District'].apply(format_house_district)

In [11]:
#ensure all districts have the same number of values
data['Formatted_District'].value_counts()


Formatted_District
AL-House-01     65
AL-House-54     65
AL-House-78     65
AL-House-77     65
AL-House-76     65
                ..
AL-House-33     65
AL-House-32     65
AL-House-31     65
AL-House-30     65
AL-House-105    65
Name: count, Length: 105, dtype: int64

In [12]:
# Pivot the dataframe separately into two dataframes with 'PRF_ESTIMATE' (raw numbers) and 'PCT_ESTIMATE' (%'s)
prf_estimate_df = data.pivot_table(index=['Formatted_District', 'GEOID'], columns='TITLE', values='PRF_ESTIMATE')
pct_estimate_df = data.pivot_table(index=['Formatted_District', 'GEOID'], columns='TITLE', values='PCT_ESTIMATE')

# Reset the column names after pivot
prf_estimate_df.columns = [f'{col}' for col in prf_estimate_df.columns]
pct_estimate_df.columns = [f'{col}' for col in pct_estimate_df.columns]

# Reset the index
prf_estimate_df.reset_index(inplace=True)
pct_estimate_df.reset_index(inplace=True)

prf_estimate_df

,Formatted_District,GEOID,"CITIZEN, VOTING AGE POPULATION - Citizen, 18 and over population","CITIZEN, VOTING AGE POPULATION - Female","CITIZEN, VOTING AGE POPULATION - Male",HISPANIC OR LATINO AND RACE - Cuban,HISPANIC OR LATINO AND RACE - Hispanic or Latino (of any race),HISPANIC OR LATINO AND RACE - Mexican,HISPANIC OR LATINO AND RACE - Other Hispanic or Latino,HISPANIC OR LATINO AND RACE - Puerto Rican,...,Total population - 65 years and over - Male,Total population - 65 years and over - Sex ratio (males per 100 females),Total population - 75 to 84 years,Total population - 85 years and over,Total population - Female,Total population - Male,Total population - Median age (years),Total population - Sex ratio (males per 100 females),Total population - Under 18 years,Total population - Under 5 years
0,AL-House-01,620L800US01001,38273.0,20304.0,17969.0,17.0,1306.0,986.0,128.0,175.0,...,4152.0,74.3,3007.0,1089.0,25165.0,23332.0,39.3,92.7,9729.0,2386.0
1,AL-House-02,620L800US01002,39071.0,19637.0,19434.0,253.0,1635.0,1191.0,174.0,17.0,...,4305.0,80.7,3091.0,750.0,25267.0,24670.0,43.9,97.6,10308.0,2369.0
2,AL-House-03,620L800US01003,38026.0,20555.0,17471.0,11.0,2230.0,1711.0,466.0,42.0,...,3894.0,74.9,2923.0,868.0,25895.0,23480.0,37.7,90.7,10378.0,2872.0
3,AL-House-04,620L800US01004,34946.0,18067.0,16879.0,74.0,3130.0,2016.0,613.0,427.0,...,2743.0,64.3,2402.0,522.0,24335.0,23919.0,39.3,98.3,12382.0,3107.0
4,AL-House-05,620L800US01005,37786.0,19313.0,18473.0,62.0,3662.0,2519.0,972.0,109.0,...,4043.0,86.7,2787.0,904.0,24471.0,23925.0,41.8,97.8,9868.0,2767.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,AL-House-95,620L800US01095,39917.0,21169.0,18748.0,37.0,1578.0,899.0,574.0,68.0,...,6018.0,85.6,3666.0,1326.0,25867.0,22316.0,51.2,86.3,7391.0,1012.0
101,AL-House-96,620L800US01096,33319.0,17412.0,15907.0,26.0,1471.0,487.0,497.0,461.0,...,3346.0,84.0,2502.0,687.0,24125.0,22339.0,37.6,92.6,12368.0,3671.0
102,AL-House-97,620L800US01097,35053.0,18404.0,16649.0,27.0,464.0,85.0,292.0,60.0,...,3147.0,74.5,2052.0,767.0,23491.0,22338.0,38.9,95.1,10566.0,3052.0
103,AL-House-98,620L800US01098,36305.0,19910.0,16395.0,6.0,556.0,58.0,251.0,241.0,...,3910.0,80.2,2786.0,860.0,26268.0,22313.0,40.0,84.9,12161.0,3588.0


In [13]:
#LOAD FUNCTION

def load_and_rename_data(csv):
    data = pd.read_csv(csv, dtype={'Line Number': str})
    data.rename(columns={
            'GEONAME': 'District',
            'Line Number': 'PROFLN',
            'Title': 'TITLE',
            'Estimate': 'PRF_ESTIMATE',
            'Percent Estimate':'PCT_ESTIMATE', 
            'Margin of Error': 'PRF_MG_ERROR',
            'Percent Margin of Error': 'PCT_MG_ERROR'
             }, inplace=True)
     # Filter out rows where 'District' mentions "not defined"
    data = data[~data['District'].str.contains("not defined", case=False, na=False)]
    return data

In [14]:
## translate PROFLN number into associated title
def translate_profln(profln, title_dict):
    # split the PROFLN value into keys by '-' for multi-labeled categories
    keys = profln.split('-')
    # translate each key using the title_dict
    titles = [str(title_dict.get(key, '')) for key in keys]
    # concatenate the translated titles
    translated_title = ' - '.join(title for title in titles if title)

    return translated_title

In [15]:
def update_titles_demo(data):
    # Create the title_dict
    title_dict = dict(zip(data['PROFLN'], data['TITLE']))
    # Update PROFLN values
    for i in range(2, 25):
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '1-' + str(i)
    for i in range(25, 29):
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '1-25' + ('' if i == 25 else '-' + str(i))
    for i in range(29, 33):
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '1-29' + ('' if i == 29 else '-' + str(i))
    for i in range(33, 37):
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '32.5-' + str(i)
    for i in range(37, 58):
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '32.5-36-' + str(i)
    for i in range(58, 63):
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '32.5-58' + ('' if i == 58 else '-' + str(i))
    for i in range(63, 70):
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '32.5-62.5-' + str(i)
    for i in range(70, 86):
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '69.5-' + str(i)
    for i in range(87, 90):
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '86.5-' + str(i)

    # Update TITLE values
    data['TITLE'] = data['PROFLN'].apply(lambda x: translate_profln(x, title_dict)) #using translate_profln function

    return data


In [16]:
# CLEAN UP DATA AND DROP REDUNDANT INFO FUNCTION

def clean_up_data_demo(data):
    # drop unneccessary rows and rows with placeholders
    invalid_rows = data['PRF_ESTIMATE'].isin(['N', '(X)']) | data['PRF_ESTIMATE'].isna()
    data = data[~invalid_rows]

    # The rows associated with these index markers (profln) contain duplicated data
    profln_to_remove = ['1-29', '1-25', '32.5-33', '32.5-36', '32.5-58', 
                        '32.5-62.5-63', '32.5-62.5-64', '32.5-62.5-65', 
                        '32.5-62.5-66', '32.5-62.5-67', '32.5-62.5-68', 
                        '32.5-62.5-69', '69.5-70', '69.5-76', '69.5-77', 
                        '69.5-78', '69.5-79', '69.5-80', '69.5-81', '69.5-82']
    data = data[~data['PROFLN'].isin(profln_to_remove)]

    # Delete the margin of error columns
    data = data.drop(columns=['PRF_MG_ERROR', 'PCT_MG_ERROR'])

    # Replace '(X)' values in PCT_ESTIMATE with their PRF_ESTIMATE values (usually, average and median values)
    data.loc[data['PCT_ESTIMATE'] == '(X)', 'PCT_ESTIMATE'] = data['PRF_ESTIMATE']

    # convert to numeric
    data['PCT_ESTIMATE'] = data['PCT_ESTIMATE'].str.replace(',', '').astype(float)
    data['PRF_ESTIMATE'] = data['PRF_ESTIMATE'].str.replace(',', '').astype(float)
    
    return data


In [17]:
# SHORTHAND STATE-DISTRICT FUNCTION

def format_districts(data):
    # dictionary of state names to abbreviations
    states = {
        'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA',
        'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA',
        'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA',
        'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD',
        'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
        'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH',
        'New Jersey': 'NJ', 'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC',
        'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK', 'Oregon': 'OR', 'Pennsylvania': 'PA',
        'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD', 'Tennessee': 'TN',
        'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA',
        'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'
    }

    # function to format a single district
    def format_district(district):
        # extract the state name and district number from the district string
        match = re.search(r'State House District (\d+) \(\d+\), ([A-Za-z\s]+)', district)
        if match:
            district_num, state_name = match.groups()
            district_num = district_num.zfill(2)  # Pad the district number with leading zeros if necessary
            state_abbr = states.get(state_name.strip(), state_name)  # Get the state abbreviation
            return f'{state_abbr}-House-{district_num}'  # Format it as required
        else:
            return district

    # Apply the function to the District column
    data['Formatted_District'] = data['District'].apply(format_district)

    return data


In [18]:
## TRANSPOSE DATAFRAME FUNCTION
def reorient_dfs(data):
    # Pivot the dataframe separately for 'PRF_ESTIMATE' and 'PCT_ESTIMATE'
    prf_estimate_df = data.pivot_table(index=['Formatted_District', 'GEOID'], columns='TITLE', values='PRF_ESTIMATE')
    pct_estimate_df = data.pivot_table(index=['Formatted_District', 'GEOID'], columns='TITLE', values='PCT_ESTIMATE')

    # Reset the column names after pivot
    prf_estimate_df.columns = [f'{col}' for col in prf_estimate_df.columns]
    pct_estimate_df.columns = [f'{col}' for col in pct_estimate_df.columns]

    # Reset the index
    prf_estimate_df.reset_index(inplace=True)
    pct_estimate_df.reset_index(inplace=True)

    return prf_estimate_df, pct_estimate_df


## Process all 49 states demographic characteristics

In [19]:
## PROCESS ALL 49 STATES FUNCTION
def process_demographics(file_state_list):
    prf_dfs = []
    pct_dfs = []
    
    for file_state in file_state_list:
        csv, state = file_state

        data = load_and_rename_data(csv)
        data = update_titles_demo(data)
        data = clean_up_data_demo(data)
        data = format_districts(data)

        prf_estimate_df, pct_estimate_df = reorient_dfs(data)

        prf_dfs.append(prf_estimate_df)
        pct_dfs.append(pct_estimate_df)
    
    df_prfdemo_all = pd.concat(prf_dfs, ignore_index=True)
    df_pctdemo_all = pd.concat(pct_dfs, ignore_index=True)
    
    return df_prfdemo_all, df_pctdemo_all


### Files below do not include Nebraska because they have a unicameral legislature, which was included in the lower Chamber State Legislative District (SLDL) processing in previous notebook

In [20]:
file_state_list = [
 ('data/lower/ALLSLDL_DP05_AK.csv', 'AK'),('data/lower/ALLSLDL_DP05_AL.csv', 'AL'),('data/lower/ALLSLDL_DP05_AR.csv', 'AR'),
 ('data/lower/ALLSLDL_DP05_AZ.csv', 'AZ'),('data/lower/ALLSLDL_DP05_CA.csv', 'CA'),('data/lower/ALLSLDL_DP05_CO.csv', 'CO'),
 ('data/lower/ALLSLDL_DP05_CT.csv', 'CT'),('data/lower/ALLSLDL_DP05_DE.csv', 'DE'),('data/lower/ALLSLDL_DP05_FL.csv', 'FL'),
 ('data/lower/ALLSLDL_DP05_GA.csv', 'GA'),('data/lower/ALLSLDL_DP05_HI.csv', 'HI'),('data/lower/ALLSLDL_DP05_IA.csv', 'IA'),
 ('data/lower/ALLSLDL_DP05_ID.csv', 'ID'),('data/lower/ALLSLDL_DP05_IL.csv', 'IL'),('data/lower/ALLSLDL_DP05_IN.csv', 'IN'),
 ('data/lower/ALLSLDL_DP05_KS.csv', 'KS'),('data/lower/ALLSLDL_DP05_KY.csv', 'KY'),('data/lower/ALLSLDL_DP05_LA.csv', 'LA'),
 ('data/lower/ALLSLDL_DP05_MA.csv', 'MA'),('data/lower/ALLSLDL_DP05_MD.csv', 'MD'),('data/lower/ALLSLDL_DP05_ME.csv', 'ME'),
 ('data/lower/ALLSLDL_DP05_MI.csv', 'MI'),('data/lower/ALLSLDL_DP05_MN.csv', 'MN'),('data/lower/ALLSLDL_DP05_MO.csv', 'MO'),
 ('data/lower/ALLSLDL_DP05_MS.csv', 'MS'),('data/lower/ALLSLDL_DP05_MT.csv', 'MT'),('data/lower/ALLSLDL_DP05_NC.csv', 'NC'),
 ('data/lower/ALLSLDL_DP05_ND.csv', 'ND'),('data/lower/ALLSLDL_DP05_NH.csv', 'NH'),
 ('data/lower/ALLSLDL_DP05_NJ.csv', 'NJ'),('data/lower/ALLSLDL_DP05_NM.csv', 'NM'),('data/lower/ALLSLDL_DP05_NV.csv', 'NV'),
 ('data/lower/ALLSLDL_DP05_NY.csv', 'NY'),('data/lower/ALLSLDL_DP05_OH.csv', 'OH'),('data/lower/ALLSLDL_DP05_OK.csv', 'OK'),
 ('data/lower/ALLSLDL_DP05_OR.csv', 'OR'),('data/lower/ALLSLDL_DP05_PA.csv', 'PA'),('data/lower/ALLSLDL_DP05_RI.csv', 'RI'),
 ('data/lower/ALLSLDL_DP05_SC.csv', 'SC'),('data/lower/ALLSLDL_DP05_SD.csv', 'SD'),('data/lower/ALLSLDL_DP05_TN.csv', 'TN'),
 ('data/lower/ALLSLDL_DP05_TX.csv', 'TX'),('data/lower/ALLSLDL_DP05_UT.csv', 'UT'),('data/lower/ALLSLDL_DP05_VA.csv', 'VA'),
 ('data/lower/ALLSLDL_DP05_VT.csv', 'VT'),('data/lower/ALLSLDL_DP05_WA.csv', 'WA'),('data/lower/ALLSLDL_DP05_WI.csv', 'WI'),
 ('data/lower/ALLSLDL_DP05_WV.csv', 'WV'),('data/lower/ALLSLDL_DP05_WY.csv', 'WY')
]


df_prfdemo_all, df_pctdemo_all = process_demographics(file_state_list)


In [21]:
df_prfdemo_all.columns

Index(['Formatted_District', 'GEOID',
       'CITIZEN, VOTING AGE POPULATION - Citizen, 18 and over population',
       'CITIZEN, VOTING AGE POPULATION - Female',
       'CITIZEN, VOTING AGE POPULATION - Male',
       'HISPANIC OR LATINO AND RACE - Cuban',
       'HISPANIC OR LATINO AND RACE - Hispanic or Latino (of any race)',
       'HISPANIC OR LATINO AND RACE - Mexican',
       'HISPANIC OR LATINO AND RACE - Other Hispanic or Latino',
       'HISPANIC OR LATINO AND RACE - Puerto Rican',
       'HISPANIC OR LATINO AND RACE - Two or more races',
       'HISPANIC OR LATINO AND RACE - Two races excluding Some other race, and Three or more races',
       'HISPANIC OR LATINO AND RACE - Two races including Some other race',
       'RACE - One race',
       'RACE - One race - American Indian and Alaska Native',
       'RACE - One race - Asian', 'RACE - One race - Asian Indian',
       'RACE - One race - Black or African American',
       'RACE - One race - Chamorro', 'RACE - One race - Chi

In [22]:
df_pctdemo_all.shape

(4834, 67)

# Housing Characteristics

In [23]:
## LOAD_AND_RENAME_DATA FUNCTION REMAINS THE SAME
AL_housing_df = load_and_rename_data('data/lower/ALLSLDL_DP04_AL.csv')

In [24]:
#Profln values and their corresponding TITLE/associations have changed. Revamping title formatting
def update_titles_housing(data):
    # Create the title_dict
    title_dict = dict(zip(data['PROFLN'], data['TITLE']))
    
    #Update PROFLN numbers by category
    for i in range(1, 6): 
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '0.8-' + str(i) # "HOUSING OCCUPANCY-" Category
    for i in range(6, 16): 
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '5.5-' + str(i) # "UNITS IN STRUCTURE-" Category
    for i in range(16, 27): 
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '15.5-' + str(i) # "YEAR STRUCTURE BUILT-" Category
    for i in range(27, 38): 
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '26.5-' + str(i) # "ROOMS-" Category
    for i in range(39, 45): 
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '37.5-' + str(i) # "BEDROOMS-" Category
    for i in range(45, 57): 
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '44.5-' + str(i) # "HOUSING TENURE-" Category
    for i in range(57, 62): 
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '56.5-' + str(i) # "VEHICLES AVAILABLE-" Category
    for i in range(62, 72): 
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '61.5-' + str(i) # "HOUSE HEATING FUEL-" Category
    for i in range(72, 76): 
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '71.5-' + str(i) # "SELECTED CHARACTERISTICS-" Category
    for i in range(76, 80): 
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '75.5-' + str(i) # "OCCUPANTS PER ROOM-" Category
    for i in range(80, 90): 
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '79.5-' + str(i)  # "VALUE-" Category
    for i in range(90, 93): 
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '89.5-' + str(i)   # "MORTGAGE STATUS-" Category
    for i in range(126, 136):
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '125.5-' + str(i) # "GROSS RENT-" Category
        
    ###LONG TITLES BELOW###
    
    # "SELECTED MONTHLY OWNER COSTS (SMOC)- Housing units with a mortgage- " Category
    for i in range(93, 102):
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '92.5-93' + ('' if i == 93 else '-' + str(i))
    # "SELECTED MONTHLY OWNER COSTS (SMOC)- Housing units without a mortgage- " Category
    for i in range(102, 110):
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '92.5-102' + ('' if i == 102 else '-' + str(i))  
    # "SELECTED MONTHLY OWNER COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME (SMOCAPI)- 
    # Housing units with a mortgage (excluding units where SMOCAPI cannot be computed)-" Category
    for i in range(110, 116):
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '109.5-110' + ('' if i == 110 else '-' + str(i))  
    # "SELECTED MONTHLY OWNER COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME (SMOCAPI)-
    # Housing unit without a mortgage (excluding units where SMOCAPI cannot be computed)-" Category
    for i in range(117, 125):
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '109.5-117' + ('' if i == 117 else '-' + str(i))
    #"GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME (GRAPI)-" Category
    for i in range(136, 143):
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '135.5-' + str(i)
        
    # Translate TITLE values using previously defined 'translate_profln' function
    data['TITLE'] = data['PROFLN'].apply(lambda x: translate_profln(x, title_dict))

    return data

In [25]:
AL_housing_df = update_titles_housing(AL_housing_df)

In [38]:
# Temporary settings to display more data
with pd.option_context('display.max_rows', None):
    print(AL_housing_df['TITLE'].unique())


['HOUSING OCCUPANCY - Total housing units'
 'HOUSING OCCUPANCY - Occupied housing units'
 'HOUSING OCCUPANCY - Vacant housing units'
 'HOUSING OCCUPANCY - Homeowner vacancy rate'
 'HOUSING OCCUPANCY - Rental vacancy rate'
 'UNITS IN STRUCTURE - 1-unit, detached'
 'UNITS IN STRUCTURE - 1-unit, attached' 'UNITS IN STRUCTURE - 2 units'
 'UNITS IN STRUCTURE - 3 or 4 units' 'UNITS IN STRUCTURE - 5 to 9 units'
 'UNITS IN STRUCTURE - 10 to 19 units'
 'UNITS IN STRUCTURE - 20 or more units'
 'UNITS IN STRUCTURE - Mobile home'
 'UNITS IN STRUCTURE - Boat, RV, van, etc.'
 'YEAR STRUCTURE BUILT - Built 2020 or later'
 'YEAR STRUCTURE BUILT - Built 2010 to 2019'
 'YEAR STRUCTURE BUILT - Built 2000 to 2009'
 'YEAR STRUCTURE BUILT - Built 1990 to 1999'
 'YEAR STRUCTURE BUILT - Built 1980 to 1989'
 'YEAR STRUCTURE BUILT - Built 1970 to 1979'
 'YEAR STRUCTURE BUILT - Built 1960 to 1969'
 'YEAR STRUCTURE BUILT - Built 1950 to 1959'
 'YEAR STRUCTURE BUILT - Built 1940 to 1949'
 'YEAR STRUCTURE BUILT - B

In [27]:
# CLEAN UP DATA AND DROP REDUNDANT INFO FROM HOUSING DATASET FUNCTION

def clean_up_data_housing(data):
    # drop unneccessary rows and rows with placeholders
    invalid_rows = data['PRF_ESTIMATE'].isin(['N', '(X)']) | data['PRF_ESTIMATE'].isna()
    data = data[~invalid_rows]

    # Drop rows where 'TITLE' contains 'Not computed'
    data = data[~data['TITLE'].str.contains('Not computed', na=False)]

    # The rows associated with these (profln) contain duplicated data
    profln_to_remove = ['5.5-6','15.5-16', '26.5-27', '38',                        # Duplicated Total Housing Units
                        '44.5-45','44.5-50', '56.5-57', '61.5-62','71.5-72', '75.5-76', #Dup. Occupied HU
                        '79.5-80', '89.5-90', '92.5-93','109.5-110','92.5-102', '109.5-117', #Mortgage/No Mortgage
                        '135.5-136']                                                       #Dup. HU Renting
    
    data = data[~data['PROFLN'].isin(profln_to_remove)]

    # Delete the margin of error columns
    data = data.drop(columns=['PRF_MG_ERROR', 'PCT_MG_ERROR'])


    # Replace '(X)' values in PCT_ESTIMATE with their PRF_ESTIMATE values (usually average and median values)
    data.loc[data['PCT_ESTIMATE'] == '(X)', 'PCT_ESTIMATE'] = data['PRF_ESTIMATE']
     

    # Remove '+' and ',' from 'PCT_ESTIMATE' and 'PRF_ESTIMATE', then convert to float
    for col in ['PCT_ESTIMATE', 'PRF_ESTIMATE']:
        data[col] = data[col].str.replace(',', '').str.replace('+', '').astype(float)
    
    return data



In [28]:
AL_housing_df = clean_up_data_housing(AL_housing_df)

In [29]:
format_districts(AL_housing_df)

,GEOID,District,PROFLN,TITLE,PRF_ESTIMATE,PCT_ESTIMATE,Formatted_District
3,620L800US01001,"State House District 1 (2022), Alabama",0.8-1,HOUSING OCCUPANCY - Total housing units,23127.0,23127.0,AL-House-01
4,620L800US01001,"State House District 1 (2022), Alabama",0.8-2,HOUSING OCCUPANCY - Occupied housing units,19955.0,86.3,AL-House-01
5,620L800US01001,"State House District 1 (2022), Alabama",0.8-3,HOUSING OCCUPANCY - Vacant housing units,3172.0,13.7,AL-House-01
7,620L800US01001,"State House District 1 (2022), Alabama",0.8-4,HOUSING OCCUPANCY - Homeowner vacancy rate,1.7,1.7,AL-House-01
8,620L800US01001,"State House District 1 (2022), Alabama",0.8-5,HOUSING OCCUPANCY - Rental vacancy rate,3.7,3.7,AL-House-01
...,...,...,...,...,...,...,...
19523,620L800US01105,"State House District 105 (2022), Alabama",135.5-138,GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME (GRAPI) - 15.0 to 19.9 percent,435.0,18.2,AL-House-105
19524,620L800US01105,"State House District 105 (2022), Alabama",135.5-139,GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME (GRAPI) - 20.0 to 24.9 percent,269.0,11.3,AL-House-105
19525,620L800US01105,"State House District 105 (2022), Alabama",135.5-140,GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME (GRAPI) - 25.0 to 29.9 percent,130.0,5.4,AL-House-105
19526,620L800US01105,"State House District 105 (2022), Alabama",135.5-141,GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME (GRAPI) - 30.0 to 34.9 percent,200.0,8.4,AL-House-105


In [30]:
AL_housing_prf, AL_housing_pct = reorient_dfs(AL_housing_df)

In [31]:
AL_housing_pct

,Formatted_District,GEOID,BEDROOMS - 1 bedroom,BEDROOMS - 2 bedrooms,BEDROOMS - 3 bedrooms,BEDROOMS - 4 bedrooms,BEDROOMS - 5 or more bedrooms,BEDROOMS - No bedroom,"GROSS RENT - $1,000 to $1,499","GROSS RENT - $1,500 to $1,999",...,YEAR STRUCTURE BUILT - Built 1939 or earlier,YEAR STRUCTURE BUILT - Built 1940 to 1949,YEAR STRUCTURE BUILT - Built 1950 to 1959,YEAR STRUCTURE BUILT - Built 1960 to 1969,YEAR STRUCTURE BUILT - Built 1970 to 1979,YEAR STRUCTURE BUILT - Built 1980 to 1989,YEAR STRUCTURE BUILT - Built 1990 to 1999,YEAR STRUCTURE BUILT - Built 2000 to 2009,YEAR STRUCTURE BUILT - Built 2010 to 2019,YEAR STRUCTURE BUILT - Built 2020 or later
0,AL-House-01,620L800US01001,7.0,25.8,50.2,12.3,4.0,0.7,8.5,2.9,...,5.6,3.7,10.4,14.8,16.8,12.9,18.6,10.1,7.0,0.1
1,AL-House-02,620L800US01002,3.7,21.7,50.3,19.4,4.1,0.9,16.3,3.1,...,2.4,2.4,4.8,12.8,15.4,14.2,17.3,17.7,12.9,0.0
2,AL-House-03,620L800US01003,8.2,27.1,47.6,11.4,3.2,2.5,8.9,1.2,...,8.4,6.5,13.0,17.8,17.5,9.2,9.1,10.0,8.4,0.1
3,AL-House-04,620L800US01004,7.4,17.5,39.4,25.4,8.8,1.5,35.1,2.7,...,5.8,1.8,8.9,10.1,10.1,10.4,13.3,20.4,18.5,0.6
4,AL-House-05,620L800US01005,3.7,21.8,50.7,18.4,3.4,2.0,15.9,3.1,...,2.9,2.2,7.7,9.1,14.1,16.7,16.6,17.9,12.5,0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,AL-House-95,620L800US01095,6.6,28.6,48.8,11.5,2.9,1.6,45.6,17.6,...,0.7,0.4,1.7,3.4,6.0,18.8,25.3,34.7,8.7,0.2
101,AL-House-96,620L800US01096,8.7,14.0,40.4,31.4,4.6,0.9,41.3,26.9,...,1.4,1.1,2.0,6.1,10.8,16.4,22.6,21.2,17.9,0.3
102,AL-House-97,620L800US01097,14.6,30.6,42.3,8.4,1.6,2.5,23.0,1.9,...,17.1,14.2,19.8,19.4,13.8,9.0,3.1,2.5,1.1,0.1
103,AL-House-98,620L800US01098,4.9,19.8,56.2,16.4,2.3,0.4,34.8,4.6,...,2.8,4.1,16.5,19.5,22.0,13.0,9.7,8.8,3.6,0.1


In [32]:
def process_housing(file_state_list):
    prf_dfs = []
    pct_dfs = []
    error_files = []

    for file_state in file_state_list:
        csv, state = file_state
        try:
            data = load_and_rename_data(csv)
            data = update_titles_housing(data)
            data = clean_up_data_housing(data)
            data = format_districts(data)

            prf_estimate_df, pct_estimate_df = reorient_dfs(data)

            prf_dfs.append(prf_estimate_df)
            pct_dfs.append(pct_estimate_df)

        except ValueError as e:
            print(f"Error processing file: {csv}")
            error_files.append((csv, str(e)))
            # Optionally, print or log the rows causing the error
            problematic_rows = data[(data['PCT_ESTIMATE'].str.contains('-', na=False)) |
                                    (data['PRF_ESTIMATE'].str.contains('-', na=False))]
            print(problematic_rows)

    df_prfhousing_all = pd.concat(prf_dfs, ignore_index=True) if prf_dfs else pd.DataFrame()
    df_pcthousing_all = pd.concat(pct_dfs, ignore_index=True) if pct_dfs else pd.DataFrame()

    return df_prfhousing_all, df_pcthousing_all, error_files

file_state_list = [
 ('data/lower/ALLSLDL_DP04_AK.csv', 'AK'),('data/lower/ALLSLDL_DP04_AL.csv', 'AL'),('data/lower/ALLSLDL_DP04_AR.csv', 'AR'),
 ('data/lower/ALLSLDL_DP04_AZ.csv', 'AZ'),('data/lower/ALLSLDL_DP04_CA.csv', 'CA'),('data/lower/ALLSLDL_DP04_CO.csv', 'CO'),
 ('data/lower/ALLSLDL_DP04_CT.csv', 'CT'),('data/lower/ALLSLDL_DP04_DE.csv', 'DE'),('data/lower/ALLSLDL_DP04_FL.csv', 'FL'),
 ('data/lower/ALLSLDL_DP04_GA.csv', 'GA'),('data/lower/ALLSLDL_DP04_HI.csv', 'HI'),('data/lower/ALLSLDL_DP04_IA.csv', 'IA'),
 ('data/lower/ALLSLDL_DP04_ID.csv', 'ID'),('data/lower/ALLSLDL_DP04_IL.csv', 'IL'),('data/lower/ALLSLDL_DP04_IN.csv', 'IN'),
 ('data/lower/ALLSLDL_DP04_KS.csv', 'KS'),('data/lower/ALLSLDL_DP04_KY.csv', 'KY'),('data/lower/ALLSLDL_DP04_LA.csv', 'LA'),
 ('data/lower/ALLSLDL_DP04_MA.csv', 'MA'),('data/lower/ALLSLDL_DP04_MD.csv', 'MD'),('data/lower/ALLSLDL_DP04_ME.csv', 'ME'),
 ('data/lower/ALLSLDL_DP04_MI.csv', 'MI'),('data/lower/ALLSLDL_DP04_MN.csv', 'MN'),('data/lower/ALLSLDL_DP04_MO.csv', 'MO'),
 ('data/lower/ALLSLDL_DP04_MS.csv', 'MS'),('data/lower/ALLSLDL_DP04_MT.csv', 'MT'),('data/lower/ALLSLDL_DP04_NC.csv', 'NC'),
 ('data/lower/ALLSLDL_DP04_ND.csv', 'ND'),('data/lower/ALLSLDL_DP04_NH.csv', 'NH'),
 ('data/lower/ALLSLDL_DP04_NJ.csv', 'NJ'),('data/lower/ALLSLDL_DP04_NM.csv', 'NM'),('data/lower/ALLSLDL_DP04_NV.csv', 'NV'),
 ('data/lower/ALLSLDL_DP04_NY.csv', 'NY'),('data/lower/ALLSLDL_DP04_OH.csv', 'OH'),('data/lower/ALLSLDL_DP04_OK.csv', 'OK'),
 ('data/lower/ALLSLDL_DP04_OR.csv', 'OR'),('data/lower/ALLSLDL_DP04_PA.csv', 'PA'),('data/lower/ALLSLDL_DP04_RI.csv', 'RI'),
 ('data/lower/ALLSLDL_DP04_SC.csv', 'SC'),('data/lower/ALLSLDL_DP04_SD.csv', 'SD'),('data/lower/ALLSLDL_DP04_TN.csv', 'TN'),
 ('data/lower/ALLSLDL_DP04_TX.csv', 'TX'),('data/lower/ALLSLDL_DP04_UT.csv', 'UT'),('data/lower/ALLSLDL_DP04_VA.csv', 'VA'),
 ('data/lower/ALLSLDL_DP04_VT.csv', 'VT'),('data/lower/ALLSLDL_DP04_WA.csv', 'WA'),('data/lower/ALLSLDL_DP04_WI.csv', 'WI'),
 ('data/lower/ALLSLDL_DP04_WV.csv', 'WV'),('data/lower/ALLSLDL_DP04_WY.csv', 'WY')
]


df_prfhousing_all, df_pcthousing_all, error_files = process_housing(file_state_list)

if error_files:
    print("Files with errors:", error_files)

Error processing file: data/lower/ALLSLDL_DP04_ME.csv
                GEOID                                District     PROFLN  \
23422  620L800US23126  State House District 126 (2022), Maine  125.5-134   

                               TITLE PRF_ESTIMATE PRF_MG_ERROR PCT_ESTIMATE  \
23422  GROSS RENT - Median (dollars)            -           **          (X)   

      PCT_MG_ERROR  
23422          (X)  
Error processing file: data/lower/ALLSLDL_DP04_VT.csv
               GEOID                                           District  \
3520  620L800US50C-7  Chittenden-7 State House District (2022), Vermont   

         PROFLN                          TITLE PRF_ESTIMATE PRF_MG_ERROR  \
3520  125.5-134  GROSS RENT - Median (dollars)            -           **   

     PCT_ESTIMATE PCT_MG_ERROR  
3520          (X)          (X)  
Files with errors: [('data/lower/ALLSLDL_DP04_ME.csv', "could not convert string to float: '-'"), ('data/lower/ALLSLDL_DP04_VT.csv', "could not convert string to float:

#### Looking at the context of the values, I assessed the median rent to be approximately as below, but really given all the other features, one entry even if hugely off shouldn't affect the outcome too severely. Also have knowledge of the top features influencing PVI from my last project and median rent didn't make the top 45 democratic or republican featueres. I had to redefine the clean_up_data_housing function to make these two values 1500

In [33]:
# Same function but replace the two median rent entries with 1500
def clean_up_data_housing(data):
    # drop unneccessary rows and rows with placeholders
    invalid_rows = data['PRF_ESTIMATE'].isin(['N', '(X)']) | data['PRF_ESTIMATE'].isna()
    data = data[~invalid_rows]

    # Drop rows where 'TITLE' contains 'Not computed'
    data = data[~data['TITLE'].str.contains('Not computed', na=False)]

    # The rows associated with these (profln) contain duplicated data
    profln_to_remove = ['5.5-6','15.5-16', '26.5-27', '38',                        # Duplicated Total Housing Units
                        '44.5-45','44.5-50', '56.5-57', '61.5-62','71.5-72', '75.5-76', #Dup. Occupied HU
                        '79.5-80', '89.5-90', '92.5-93','109.5-110','92.5-102', '109.5-117', #Mortgage/No Mortgage
                        '135.5-136']                                                       #Dup. HU Renting
    
    data = data[~data['PROFLN'].isin(profln_to_remove)]

    # Delete the margin of error columns
    data = data.drop(columns=['PRF_MG_ERROR', 'PCT_MG_ERROR'])

    # Replace '-' in PCT_ESTIMATE with 1500 (inspection of data shows the only two '-' value errors dealt with median rent)
    data.loc[data['PRF_ESTIMATE'] == '-', 'PRF_ESTIMATE'] = '1500'
    # Replace '(X)' values in PCT_ESTIMATE with their PRF_ESTIMATE values (usually average and median values)
    data.loc[data['PCT_ESTIMATE'] == '(X)', 'PCT_ESTIMATE'] = data['PRF_ESTIMATE']

    # Remove '+' and ',' from 'PCT_ESTIMATE' and 'PRF_ESTIMATE', then convert to float
    for col in ['PCT_ESTIMATE', 'PRF_ESTIMATE']:
        data[col] = data[col].str.replace(',', '').str.replace('+', '').astype(float)
    
    return data



In [34]:
def process_housing(file_state_list):
    prf_dfs = []
    pct_dfs = []
    error_files = []

    for file_state in file_state_list:
        csv, state = file_state
        try:
            data = load_and_rename_data(csv)
            data = update_titles_housing(data)
            data = clean_up_data_housing(data)
            data = format_districts(data)

            prf_estimate_df, pct_estimate_df = reorient_dfs(data)

            prf_dfs.append(prf_estimate_df)
            pct_dfs.append(pct_estimate_df)

        except ValueError as e:
            print(f"Error processing file: {csv}")
            error_files.append((csv, str(e)))
            # print the rows causing an error
            problematic_rows = data[(data['PCT_ESTIMATE'].str.contains('-', na=False)) |
                                    (data['PRF_ESTIMATE'].str.contains('-', na=False))]
            print(problematic_rows)

    df_prfhousing_all = pd.concat(prf_dfs, ignore_index=True) if prf_dfs else pd.DataFrame()
    df_pcthousing_all = pd.concat(pct_dfs, ignore_index=True) if pct_dfs else pd.DataFrame()

    return df_prfhousing_all, df_pcthousing_all, error_files

df_prfhousing_all, df_pcthousing_all, error_files = process_housing(file_state_list)

if error_files:
    print("Files with errors:", error_files2)

In [36]:
df_prfhousing_all.columns

Index(['Formatted_District', 'GEOID', 'BEDROOMS - 1 bedroom',
       'BEDROOMS - 2 bedrooms', 'BEDROOMS - 3 bedrooms',
       'BEDROOMS - 4 bedrooms', 'BEDROOMS - 5 or more bedrooms',
       'BEDROOMS - No bedroom', 'GROSS RENT - $1,000 to $1,499',
       'GROSS RENT - $1,500 to $1,999',
       ...
       'YEAR STRUCTURE BUILT - Built 1939 or earlier',
       'YEAR STRUCTURE BUILT - Built 1940 to 1949',
       'YEAR STRUCTURE BUILT - Built 1950 to 1959',
       'YEAR STRUCTURE BUILT - Built 1960 to 1969',
       'YEAR STRUCTURE BUILT - Built 1970 to 1979',
       'YEAR STRUCTURE BUILT - Built 1980 to 1989',
       'YEAR STRUCTURE BUILT - Built 1990 to 1999',
       'YEAR STRUCTURE BUILT - Built 2000 to 2009',
       'YEAR STRUCTURE BUILT - Built 2010 to 2019',
       'YEAR STRUCTURE BUILT - Built 2020 or later'],
      dtype='object', length=125)

In [37]:
df_prfhousing_all

,Formatted_District,GEOID,BEDROOMS - 1 bedroom,BEDROOMS - 2 bedrooms,BEDROOMS - 3 bedrooms,BEDROOMS - 4 bedrooms,BEDROOMS - 5 or more bedrooms,BEDROOMS - No bedroom,"GROSS RENT - $1,000 to $1,499","GROSS RENT - $1,500 to $1,999",...,YEAR STRUCTURE BUILT - Built 1939 or earlier,YEAR STRUCTURE BUILT - Built 1940 to 1949,YEAR STRUCTURE BUILT - Built 1950 to 1959,YEAR STRUCTURE BUILT - Built 1960 to 1969,YEAR STRUCTURE BUILT - Built 1970 to 1979,YEAR STRUCTURE BUILT - Built 1980 to 1989,YEAR STRUCTURE BUILT - Built 1990 to 1999,YEAR STRUCTURE BUILT - Built 2000 to 2009,YEAR STRUCTURE BUILT - Built 2010 to 2019,YEAR STRUCTURE BUILT - Built 2020 or later
0,AK-House-01,620L800US02001,1301.0,2601.0,2817.0,1137.0,365.0,501.0,730.0,372.0,...,956.0,433.0,893.0,676.0,1955.0,1500.0,1164.0,653.0,487.0,5.0
1,AK-House-02,620L800US02002,1619.0,2591.0,3405.0,1182.0,320.0,749.0,725.0,313.0,...,767.0,337.0,383.0,847.0,2295.0,2376.0,1436.0,1024.0,397.0,4.0
2,AK-House-03,620L800US02003,1271.0,2330.0,2985.0,899.0,198.0,383.0,758.0,349.0,...,251.0,141.0,111.0,680.0,1644.0,2148.0,1360.0,1071.0,658.0,2.0
3,AK-House-04,620L800US02004,1390.0,2945.0,2615.0,1045.0,140.0,383.0,1054.0,656.0,...,864.0,262.0,568.0,618.0,2301.0,1922.0,837.0,723.0,415.0,8.0
4,AK-House-05,620L800US02005,1096.0,2091.0,3115.0,1230.0,344.0,535.0,599.0,453.0,...,212.0,305.0,573.0,745.0,1477.0,2497.0,1318.0,883.0,393.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4829,WY-House-58,620L800US56058,420.0,979.0,1578.0,712.0,181.0,9.0,293.0,23.0,...,504.0,367.0,86.0,169.0,547.0,461.0,315.0,798.0,584.0,48.0
4830,WY-House-59,620L800US56059,274.0,1110.0,2335.0,655.0,107.0,48.0,264.0,68.0,...,308.0,324.0,1814.0,405.0,482.0,330.0,226.0,352.0,288.0,0.0
4831,WY-House-60,620L800US56060,55.0,595.0,1435.0,686.0,599.0,66.0,167.0,86.0,...,29.0,37.0,146.0,163.0,1309.0,908.0,441.0,265.0,138.0,0.0
4832,WY-House-61,620L800US56061,202.0,961.0,2282.0,714.0,162.0,72.0,195.0,328.0,...,124.0,25.0,128.0,1365.0,591.0,469.0,467.0,758.0,466.0,0.0


# Social Characteristics

In [39]:
## LOAD_AND_RENAME_DATA FUNCTION REMAINS THE SAME
AL_social_df = load_and_rename_data('data/lower/ALLSLDL_DP02_AL.csv')

In [40]:
def update_titles_social(data):
    # Create the title_dict
    title_dict = dict(zip(data['PROFLN'], data['TITLE']))

    ####### DICTIONARIES FOR PROFLN NUMBERS THAT NEED A DIRECT MAPPING#######
    # "HOUSEHOLDS BY TYPE" category
    households_by_type_dict = {
        '2': '1-2', '3': '1-2-3', '4': '1-4', '5': '1-4-5',
        '6': '1-6', '7': '1-6-7', '8': '1-6-8', '9': '1-6-9',
        '10': '1-10', '11': '1-10-11', '12': '1-10-12', '13': '1-10-13'
    }
    # "FERTILITY" category
    fertility_dict = {
        '37': '36.5-37', '38': '36.5-38', '39': '36.5-37-39', 
        '40': '36.5-37-40', '41': '36.5-37-41', '42': '36.5-37-42', 
        '43': '36.5-37-43'
    }
    # "GRANDPARENTS" category
    grandparents_dict = {
        '44': '43.5-44', '45': '43.5-44-45', '45.9': '45-45.9',
        '46': '45-45.9-46', '47': '45-45.9-47', '48': '45-45.9-48',
        '49': '45-45.9-49', '51': '50-51', '52': '50-52'
    }
    # "VETERAN STATUS" category
    veteran_status_dict = {
        '69': '68.5-69', '70': '68.5-70'
    }
    # "PLACE OF BIRTH" category
    place_of_birth_dict = {
        '88': '87.5-88', '89': '87.5-89', '90': '87.5-89-90', 
        '91': '87.5-89-91', '92': '87.5-89-92', '93': '87.5-89-93', 
        '94': '87.5-94'
    }
    # "U.S. CITIZENSHIP STATUS" category
    citizenship_status_dict = {
        '96': '95-96', '97': '95-97'
    }
    # "YEAR OF ENTRY" category
    year_of_entry_dict = {
        '99': '98-99', '100': '98-99-100', '101': '98-99-101',
        '102': '98-102', '103': '98-102-103', '104': '98-102-104'
    }
    # "LANGUAGE SPOKEN AT HOME" category
    language_spoken_dict = {
        '112': '111.5-112', '113': '111.5-113', '114': '111.5-114',
        '115': '111.5-114-115', '116': '111.5-116', '117': '111.5-114-117',
        '118': '111.5-118', '119': '111.5-118-119', '120': '111.5-120',
        '121': '111.5-120-121', '122': '111.5-114-122', '123': '111.5-114-122-123'
    }
    # "COMPUTERS AND INTERNET USE" category
    computers_and_internet_use_dict = {
        '152': '151.5-152', 
        '153': '151.5-152-153', 
        '154': '151.5-152-154'
    }
    

    ###### PROFLN NUMBERS THAT FOLLOW A SIMPLE PATTERN OVER A LONG RANGE######
    # "RELATIONSHIP" category
    for i in range(19, 25):
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '18-' + str(i)
    # "MARITAL STATUS" category
    for i in range(26, 31):
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '25-' + str(i)
    for i in range(32, 37):
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '31-' + str(i)
    # "SCHOOL ENROLLMENT" category
    for i in range(54, 59):
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '53-' + str(i)
    # "EDUCATIONAL ATTAINMENT" category
    for i in range(59, 69):
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '58.5-59' + ('' if i == 59 else '-' + str(i))
    # "DISABILITY STATUS OF THE CIVILIAN NONINSTITUTIONALIZED POPULATION" category
    for i in range(72, 79):
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '71-' + str(i)
    # "RESIDENCE 1 YEAR AGO" category
    for i in range(80, 88):
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '78.5-' + str(i)
    # "WORLD REGION OF BIRTH OF FOREIGN BORN" category
    for i in range(105, 112):
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '104.5-' + str(i)
    # "ANCESTRY" category
    for i in range(125, 152):
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '123.5-' + str(i)



    ###### DIRECT MAPPING #####
    for old, new in households_by_type_dict.items():
        data.loc[data['PROFLN'] == old, 'PROFLN'] = new
    for old, new in fertility_dict.items():
        data.loc[data['PROFLN'] == old, 'PROFLN'] = new
    for old, new in grandparents_dict.items():
        data.loc[data['PROFLN'] == old, 'PROFLN'] = new
    for old, new in veteran_status_dict.items():
        data.loc[data['PROFLN'] == old, 'PROFLN'] = new
    for old, new in place_of_birth_dict.items():
        data.loc[data['PROFLN'] == old, 'PROFLN'] = new
    for old, new in citizenship_status_dict.items():
        data.loc[data['PROFLN'] == old, 'PROFLN'] = new
    for old, new in year_of_entry_dict.items():
        data.loc[data['PROFLN'] == old, 'PROFLN'] = new
    for old, new in language_spoken_dict.items():
        data.loc[data['PROFLN'] == old, 'PROFLN'] = new
    for old, new in computers_and_internet_use_dict.items():
        data.loc[data['PROFLN'] == old, 'PROFLN'] = new

    # Translate TITLE values using previously defined 'translate_profln' function
    data['PROFLN'] = data['PROFLN'].astype(str)
    data['TITLE'] = data['PROFLN'].apply(lambda x: translate_profln(x, title_dict))



    return data


In [41]:
AL_social_df = update_titles_social(AL_social_df)

In [42]:
# CLEAN UP DATA AND DROP REDUNDANT INFO FROM SOCIAL DATASET FUNCTION

def clean_up_data_social(data):
    # drop unneccessary rows and rows with placeholders
    invalid_rows = (data['PRF_ESTIMATE'].isin(['N', '(X)']) | 
                data['PCT_ESTIMATE'].isin(['N', '(X)']) |
                data['PRF_ESTIMATE'].isna())
    data = data[~invalid_rows]

    # Drop rows where 'TITLE' contains 'Not computed'
    data = data[~data['TITLE'].str.contains('Not computed', na=False)]

    # The rows associated with these (profln) contain duplicated data
    profln_to_remove = ['50','95','98-102', 
                        '1','152','124']  # Duplicated Grandparents, Foreign born, Total households, total pop               
    
    data = data[~data['PROFLN'].isin(profln_to_remove)]

    # Delete the margin of error columns
    data = data.drop(columns=['PRF_MG_ERROR', 'PCT_MG_ERROR'])

    # Replace '(X)' values in PCT_ESTIMATE with their PRF_ESTIMATE values (usually average and median values)
    data.loc[data['PCT_ESTIMATE'] == '(X)', 'PCT_ESTIMATE'] = data['PRF_ESTIMATE']
    # Replace '-' in PCT_ESTIMATE with 0 (inspection of data shows at or close to zero but not enough samples)
    data.loc[data['PCT_ESTIMATE'] == '-', 'PCT_ESTIMATE'] = '0'

    # Remove '+' and ',' from 'PCT_ESTIMATE' and 'PRF_ESTIMATE', then convert to float
    for col in ['PCT_ESTIMATE', 'PRF_ESTIMATE']:
        data[col] = data[col].str.replace(',', '').str.replace('+', '').astype(float)
    
    return data

In [43]:
AL_social_df = clean_up_data_social(AL_social_df)

In [44]:
format_districts(AL_social_df)

,GEOID,District,PROFLN,TITLE,PRF_ESTIMATE,PCT_ESTIMATE,Formatted_District
4,620L800US01001,"State House District 1 (2022), Alabama",1-2,Total households - Married-couple household,9408.0,47.1,AL-House-01
5,620L800US01001,"State House District 1 (2022), Alabama",1-2-3,Total households - Married-couple household - With children of the householder under 18 years,3087.0,15.5,AL-House-01
6,620L800US01001,"State House District 1 (2022), Alabama",1-4,Total households - Cohabiting couple household,1037.0,5.2,AL-House-01
7,620L800US01001,"State House District 1 (2022), Alabama",1-4-5,Total households - Cohabiting couple household - With children of the householder under 18 years,219.0,1.1,AL-House-01
8,620L800US01001,"State House District 1 (2022), Alabama",1-6,"Total households - Male householder, no spouse/partner present",3248.0,16.3,AL-House-01
...,...,...,...,...,...,...,...
20993,620L800US01105,"State House District 105 (2022), Alabama",123.5-150,ANCESTRY - Welsh,872.0,1.9,AL-House-105
20994,620L800US01105,"State House District 105 (2022), Alabama",123.5-151,ANCESTRY - West Indian (excluding Hispanic origin groups),33.0,0.1,AL-House-105
20997,620L800US01105,"State House District 105 (2022), Alabama",151.5-152,COMPUTERS AND INTERNET USE - Total households,17012.0,17012.0,AL-House-105
20998,620L800US01105,"State House District 105 (2022), Alabama",151.5-152-153,COMPUTERS AND INTERNET USE - Total households - With a computer,15427.0,90.7,AL-House-105


In [45]:
AL_social_prf, AL_social_pct = reorient_dfs(AL_social_df)

In [46]:
AL_social_prf

,Formatted_District,GEOID,ANCESTRY - American,ANCESTRY - Arab,ANCESTRY - Czech,ANCESTRY - Danish,ANCESTRY - Dutch,ANCESTRY - English,ANCESTRY - French (except Basque),ANCESTRY - French Canadian,...,Total households - Married-couple household - With children of the householder under 18 years,VETERAN STATUS - Civilian population 18 years and over,VETERAN STATUS - Civilian veterans,WORLD REGION OF BIRTH OF FOREIGN BORN - Africa,WORLD REGION OF BIRTH OF FOREIGN BORN - Asia,WORLD REGION OF BIRTH OF FOREIGN BORN - Europe,"WORLD REGION OF BIRTH OF FOREIGN BORN - Foreign-born population, excluding population born at sea",WORLD REGION OF BIRTH OF FOREIGN BORN - Latin America,WORLD REGION OF BIRTH OF FOREIGN BORN - Northern America,WORLD REGION OF BIRTH OF FOREIGN BORN - Oceania
0,AL-House-01,620L800US01001,6566.0,43.0,15.0,36.0,261.0,5234.0,451.0,73.0,...,3087.0,38721.0,2912.0,4.0,293.0,43.0,704.0,292.0,55.0,17.0
1,AL-House-02,620L800US01002,7325.0,174.0,132.0,71.0,463.0,6252.0,664.0,75.0,...,3815.0,39591.0,3197.0,0.0,243.0,107.0,1062.0,698.0,14.0,0.0
2,AL-House-03,620L800US01003,10273.0,24.0,75.0,12.0,206.0,4175.0,253.0,20.0,...,2620.0,38942.0,2725.0,73.0,392.0,113.0,1475.0,875.0,22.0,0.0
3,AL-House-04,620L800US01004,5845.0,367.0,13.0,166.0,622.0,5541.0,882.0,203.0,...,4780.0,35841.0,4333.0,167.0,1016.0,360.0,3005.0,1421.0,41.0,0.0
4,AL-House-05,620L800US01005,6436.0,0.0,57.0,91.0,367.0,6080.0,729.0,120.0,...,3666.0,38479.0,3205.0,15.0,289.0,143.0,1614.0,1095.0,72.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,AL-House-95,620L800US01095,12010.0,40.0,215.0,86.0,343.0,5248.0,1415.0,222.0,...,2232.0,40792.0,4664.0,9.0,431.0,622.0,2149.0,977.0,83.0,27.0
101,AL-House-96,620L800US01096,8163.0,23.0,79.0,158.0,116.0,6164.0,1406.0,176.0,...,3934.0,33887.0,3804.0,19.0,737.0,366.0,2060.0,773.0,146.0,19.0
102,AL-House-97,620L800US01097,2794.0,104.0,26.0,14.0,192.0,3487.0,823.0,62.0,...,1831.0,35226.0,2804.0,44.0,297.0,77.0,555.0,124.0,7.0,6.0
103,AL-House-98,620L800US01098,5335.0,86.0,0.0,0.0,69.0,1642.0,718.0,18.0,...,2628.0,36348.0,2747.0,5.0,175.0,57.0,497.0,260.0,0.0,0.0


In [47]:
## PROCESS ALL 49 STATES SOCIAL FUNCTION
def process_social(file_state_list):
    prf_dfs = []
    pct_dfs = []
    
    for file_state in file_state_list:
        csv, state = file_state

        data = load_and_rename_data(csv)
        data = update_titles_social(data)
        data = clean_up_data_social(data)
        data = format_districts(data)

        prf_estimate_df, pct_estimate_df = reorient_dfs(data)

        prf_dfs.append(prf_estimate_df)
        pct_dfs.append(pct_estimate_df)
    
    df_prfsocial_all = pd.concat(prf_dfs, ignore_index=True)
    df_pctsocial_all = pd.concat(pct_dfs, ignore_index=True)
    
    return df_prfsocial_all, df_pctsocial_all


In [48]:


def process_social(file_state_list):
    prf_dfs = []
    pct_dfs = []
    error_files = []

    for file_state in file_state_list:
        csv, state = file_state
        try:
            data = load_and_rename_data(csv)
            data = update_titles_social(data)
            data = clean_up_data_social(data)
            data = format_districts(data)

            prf_estimate_df, pct_estimate_df = reorient_dfs(data)

            prf_dfs.append(prf_estimate_df)
            pct_dfs.append(pct_estimate_df)

        except ValueError as e:
            print(f"Error processing file: {csv}")
            error_files.append((csv, str(e)))
            # Optionally, print or log the rows causing the error
            problematic_rows = data[(data['PCT_ESTIMATE'].str.contains('-', na=False)) |
                                    (data['PRF_ESTIMATE'].str.contains('-', na=False))]
            print(problematic_rows)

    df_prfsocial_all = pd.concat(prf_dfs, ignore_index=True) if prf_dfs else pd.DataFrame()
    df_pctsocial_all = pd.concat(pct_dfs, ignore_index=True) if pct_dfs else pd.DataFrame()

    return df_prfsocial_all, df_pctsocial_all, error_files

# Usage
file_state_list = file_state_list = [
 ('data/lower/ALLSLDL_DP02_AK.csv', 'AK'),('data/lower/ALLSLDL_DP02_AL.csv', 'AL'),('data/lower/ALLSLDL_DP02_AR.csv', 'AR'),
 ('data/lower/ALLSLDL_DP02_AZ.csv', 'AZ'),('data/lower/ALLSLDL_DP02_CA.csv', 'CA'),('data/lower/ALLSLDL_DP02_CO.csv', 'CO'),
 ('data/lower/ALLSLDL_DP02_CT.csv', 'CT'),('data/lower/ALLSLDL_DP02_DE.csv', 'DE'),('data/lower/ALLSLDL_DP02_FL.csv', 'FL'),
 ('data/lower/ALLSLDL_DP02_GA.csv', 'GA'),('data/lower/ALLSLDL_DP02_HI.csv', 'HI'),('data/lower/ALLSLDL_DP02_IA.csv', 'IA'),
 ('data/lower/ALLSLDL_DP02_ID.csv', 'ID'),('data/lower/ALLSLDL_DP02_IL.csv', 'IL'),('data/lower/ALLSLDL_DP02_IN.csv', 'IN'),
 ('data/lower/ALLSLDL_DP02_KS.csv', 'KS'),('data/lower/ALLSLDL_DP02_KY.csv', 'KY'),('data/lower/ALLSLDL_DP02_LA.csv', 'LA'),
 ('data/lower/ALLSLDL_DP02_MA.csv', 'MA'),('data/lower/ALLSLDL_DP02_MD.csv', 'MD'),('data/lower/ALLSLDL_DP02_ME.csv', 'ME'),
 ('data/lower/ALLSLDL_DP02_MI.csv', 'MI'),('data/lower/ALLSLDL_DP02_MN.csv', 'MN'),('data/lower/ALLSLDL_DP02_MO.csv', 'MO'),
 ('data/lower/ALLSLDL_DP02_MS.csv', 'MS'),('data/lower/ALLSLDL_DP02_MT.csv', 'MT'),('data/lower/ALLSLDL_DP02_NC.csv', 'NC'),
 ('data/lower/ALLSLDL_DP02_ND.csv', 'ND'),('data/lower/ALLSLDL_DP02_NH.csv', 'NH'),
 ('data/lower/ALLSLDL_DP02_NJ.csv', 'NJ'),('data/lower/ALLSLDL_DP02_NM.csv', 'NM'),('data/lower/ALLSLDL_DP02_NV.csv', 'NV'),
 ('data/lower/ALLSLDL_DP02_NY.csv', 'NY'),('data/lower/ALLSLDL_DP02_OH.csv', 'OH'),('data/lower/ALLSLDL_DP02_OK.csv', 'OK'),
 ('data/lower/ALLSLDL_DP02_OR.csv', 'OR'),('data/lower/ALLSLDL_DP02_PA.csv', 'PA'),('data/lower/ALLSLDL_DP02_RI.csv', 'RI'),
 ('data/lower/ALLSLDL_DP02_SC.csv', 'SC'),('data/lower/ALLSLDL_DP02_SD.csv', 'SD'),('data/lower/ALLSLDL_DP02_TN.csv', 'TN'),
 ('data/lower/ALLSLDL_DP02_TX.csv', 'TX'),('data/lower/ALLSLDL_DP02_UT.csv', 'UT'),('data/lower/ALLSLDL_DP02_VA.csv', 'VA'),
 ('data/lower/ALLSLDL_DP02_VT.csv', 'VT'),('data/lower/ALLSLDL_DP02_WA.csv', 'WA'),('data/lower/ALLSLDL_DP02_WI.csv', 'WI'),
 ('data/lower/ALLSLDL_DP02_WV.csv', 'WV'),('data/lower/ALLSLDL_DP02_WY.csv', 'WY')
]
df_prfsocial_all, df_pctsocial_all, error_files = process_social(file_state_list)

if error_files:
    print("Files with errors:", error_files)


In [49]:
df_prfsocial_all

,Formatted_District,GEOID,ANCESTRY - American,ANCESTRY - Arab,ANCESTRY - Czech,ANCESTRY - Danish,ANCESTRY - Dutch,ANCESTRY - English,ANCESTRY - French (except Basque),ANCESTRY - French Canadian,...,Total households - Married-couple household - With children of the householder under 18 years,VETERAN STATUS - Civilian population 18 years and over,VETERAN STATUS - Civilian veterans,WORLD REGION OF BIRTH OF FOREIGN BORN - Africa,WORLD REGION OF BIRTH OF FOREIGN BORN - Asia,WORLD REGION OF BIRTH OF FOREIGN BORN - Europe,"WORLD REGION OF BIRTH OF FOREIGN BORN - Foreign-born population, excluding population born at sea",WORLD REGION OF BIRTH OF FOREIGN BORN - Latin America,WORLD REGION OF BIRTH OF FOREIGN BORN - Northern America,WORLD REGION OF BIRTH OF FOREIGN BORN - Oceania
0,AK-House-01,620L800US02001,468.0,43.0,76.0,60.0,226.0,1675.0,341.0,61.0,...,1155.0,13684.0,1347.0,28.0,955.0,100.0,1344.0,172.0,66.0,23.0
1,AK-House-02,620L800US02002,344.0,30.0,52.0,128.0,225.0,1393.0,484.0,159.0,...,1125.0,14351.0,1444.0,22.0,1091.0,137.0,1614.0,208.0,135.0,21.0
2,AK-House-03,620L800US02003,620.0,52.0,69.0,162.0,265.0,2054.0,322.0,80.0,...,1367.0,13107.0,1018.0,38.0,670.0,47.0,1201.0,84.0,165.0,197.0
3,AK-House-04,620L800US02004,799.0,0.0,21.0,137.0,226.0,1643.0,663.0,132.0,...,1274.0,15203.0,1145.0,23.0,1092.0,203.0,1924.0,485.0,43.0,78.0
4,AK-House-05,620L800US02005,469.0,27.0,127.0,178.0,229.0,948.0,259.0,288.0,...,1569.0,13293.0,1473.0,63.0,2519.0,56.0,3057.0,355.0,21.0,43.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4829,WY-House-58,620L800US56058,479.0,0.0,0.0,44.0,294.0,921.0,247.0,33.0,...,562.0,7062.0,645.0,0.0,0.0,45.0,134.0,50.0,39.0,0.0
4830,WY-House-59,620L800US56059,274.0,0.0,119.0,43.0,75.0,714.0,105.0,90.0,...,570.0,7239.0,512.0,0.0,0.0,21.0,205.0,184.0,0.0,0.0
4831,WY-House-60,620L800US56060,803.0,0.0,24.0,139.0,157.0,1596.0,86.0,27.0,...,802.0,6192.0,492.0,12.0,58.0,54.0,227.0,103.0,0.0,0.0
4832,WY-House-61,620L800US56061,823.0,0.0,92.0,275.0,50.0,775.0,222.0,29.0,...,835.0,7305.0,957.0,0.0,21.0,48.0,94.0,25.0,0.0,0.0


In [50]:
# Assuming 'data' is your DataFrame
nan_counts = data.isna().sum()
print(nan_counts)


GEOID                 0
District              0
PROFLN                0
TITLE                 0
PRF_ESTIMATE          0
PCT_ESTIMATE          0
Formatted_District    0
dtype: int64


# Economic Characteristics

In [51]:
AL_econ_df = load_and_rename_data('data/lower/ALLSLDL_DP03_AL.csv')

In [52]:
def update_titles_econ(data):
   
    # Shorten two long titles--
    #INCOME AND BENEFITS (IN 2021 INFLATION-ADJUSTED DOLLARS)
    data.loc[data['PROFLN'] == '50.5', 'TITLE'] = 'INCOME AND BENEFITS' 
    #PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL
    data.loc[data['PROFLN'] == '118.5', 'TITLE'] = 'INCOME BELOW POVERTY LEVEL' 
   
    # now create the title_dict
    title_dict = dict(zip(data['PROFLN'], data['TITLE']))

    ####### DICTIONARIES FOR PROFLN NUMBERS THAT NEED A DIRECT MAPPING#######
    
    # Dictionary "MISC FEMALE EMPLOYMENT"
    misc_female_employment_dict = {
        '15': '0.8-1-14-15',
        '16': '0.8-10-16',
        '17': '0.8-10-16-17'
    }
    # Dictionary "MISC INCOME AND BENEFITS" category
    misc_income_and_benefits_dict = {
        '65': '50.5-51-64-65',
        '66': '50.5-51-66',
        '67': '50.5-51-66-67',
        '68': '50.5-51-68',
        '69': '50.5-51-68-69',
        '70': '50.5-51-70',
        '71': '50.5-51-70-71',
        '72': '50.5-51-72',
        '73': '50.5-51-72-73',
        '74': '50.5-51-74',
    }
    
    # Dictionary HEALTH INSURANCE COVERAGE category
    health_insurance_coverage_dict = {
        '95': '94.5-95',
        '96': '94.5-95-96',
        '97': '94.5-95-97',
        '98': '94.5-95-98',
        '99': '94.5-95-99',
        '100': '94.5-100',
        '101': '94.5-100-101',
        '103': '102-103',
        '104': '94.5-104',
        '105': '94.5-104-105',
        '106': '94.5-104-106',
        '107': '94.5-104-107',
        '108': '94.5-104-108',
        '109': '94.5-109',
        '110': '94.5-109-110',
        '111': '94.5-109-111',
        '112': '94.5-109-112',
        '113': '94.5-109-113',
        '114': '94.5-114',
        '115': '94.5-114-115',
        '116': '94.5-114-116',
        '117': '94.5-114-117',
        '118': '94.5-114-118'
    }
    # Dictionary FAMILY POVERTY LEVEL
    family_poverty_level_dict = {
        '119': '118.5-119',
        '120': '118.5-119-120',
        '121': '118.5-119-121',
        '122': '118.5-122',
        '123': '118.5-122-123',
        '124': '118.5-122-124',
        '125': '118.5-125',
        '126': '118.5-125-126',
        '127': '118.5-125-127'
    }


    ###### PROFLN NUMBERS THAT FOLLOW A SIMPLE PATTERN OVER A LONG RANGE######
    # EMPLOYMENT STATUS category
    for i in range(1, 10):
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '0.8-1' + ('' if i == 1 else '-' + str(i))
    # "FEMALE EMPLOYMENT STATUS" category
    for i in range(10, 15):
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '0.8-10' + ('' if i == 10 else '-' + str(i))
    # "COMMUTING TO WORK" category
    for i in range(18, 26):
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '17.5-' + str(i)
    # "OCCUPATION" category
    for i in range(26, 32):
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '25.5-' + str(i)
    # "INDUSTRY" category
    for i in range(33, 46):
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '31.5-' + str(i)
    #"CLASS OF WORKER" category
    for i in range(47, 51):
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '45.5-' + str(i)
    # "INCOME AND BENEFITS" category
    for i in range(52, 65):
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '50.5-51-' + str(i)
    # FAMILY INCOME AND BENEFITS category
    for i in range(75, 88):
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '50.5-75' + ('' if i == 75 else '-' + str(i))
    # NON-FAMILY INCOME AND BENEFITS category
    for i in range(88, 95):
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '50.5-' + str(i)
    # ALL PEOPLE POVERTY LEVEL category
    for i in range (128,138):
        data.loc[data['PROFLN'] == str(i), 'PROFLN'] = '118.5-128'+ ('' if i == 128 else '-' + str(i))
    
    
    ###### DIRECT MAPPING #####
    for old, new in misc_female_employment_dict.items(): #Misc Female Employment
        data.loc[data['PROFLN'] == old, 'PROFLN'] = new
    for old, new in misc_income_and_benefits_dict.items(): #Misc income and benefits
        data.loc[data['PROFLN'] == old, 'PROFLN'] = new
    for old, new in health_insurance_coverage_dict.items(): #Health Insurance Coverage
        data.loc[data['PROFLN'] == old, 'PROFLN'] = new
    for old, new in family_poverty_level_dict.items():     # Family poverty level
        data.loc[data['PROFLN'] == old, 'PROFLN'] = new
    

    # Translate TITLE values using previously defined 'translate_profln' function
    data['PROFLN'] = data['PROFLN'].astype(str)
    data['TITLE'] = data['PROFLN'].apply(lambda x: translate_profln(x, title_dict))

    return data


In [53]:
AL_econ_df = update_titles_econ(AL_econ_df)
AL_econ_df[['PROFLN','TITLE']]

,PROFLN,TITLE
0,0,SELECTED ECONOMIC CHARACTERISTICS
1,0.5,nan
2,0.8,EMPLOYMENT STATUS
3,0.8-1,EMPLOYMENT STATUS - Population 16 years and over
4,0.8-1-2,EMPLOYMENT STATUS - Population 16 years and over - In labor force
...,...,...
17635,118.5-128-133,INCOME BELOW POVERTY LEVEL - All people - 18 years and over
17636,118.5-128-134,INCOME BELOW POVERTY LEVEL - All people - 18 to 64 years
17637,118.5-128-135,INCOME BELOW POVERTY LEVEL - All people - 65 years and over
17638,118.5-128-136,INCOME BELOW POVERTY LEVEL - All people - People in families


In [54]:
def clean_up_data_econ(data):
    
    # Replace '(X)' values in PCT_ESTIMATE with their PRF_ESTIMATE values (and vice versa)
    data.loc[data['PCT_ESTIMATE'] == '(X)', 'PCT_ESTIMATE'] = data['PRF_ESTIMATE']
    data.loc[data['PRF_ESTIMATE'] == '(X)', 'PRF_ESTIMATE'] = data['PCT_ESTIMATE']
     # Replace '-' in PCT_ESTIMATE with 0 (inspection of data shows at or close to zero but not enough samples)
    data.loc[data['PCT_ESTIMATE'] == '-', 'PCT_ESTIMATE'] = '0'
    data.loc[data['PRF_ESTIMATE'] == '-', 'PRF_ESTIMATE'] = '0'
    
    # Drop unnecessary rows and rows with placeholders
    invalid_rows = (data['PRF_ESTIMATE'].isin(['N', '(X)']) | 
                data['PCT_ESTIMATE'].isin(['N', '(X)']) |
                data['PRF_ESTIMATE'].isna())
    data = data[~invalid_rows]

    # Drop rows where 'TITLE' contains 'Not computed'
    data = data[~data['TITLE'].str.contains('Not computed', na=False)]

    # The rows associated with these (profln) contain duplicated data
    profln_to_remove = ['25.5-26', '32', '46',   # Duplicated 'Employed population 16+'
                        '0.8-1-8']               # Duplicated 'Civilian labor force'
    
    data = data[~data['PROFLN'].isin(profln_to_remove)]

    # Delete the margin of error columns
    data = data.drop(columns=['PRF_MG_ERROR', 'PCT_MG_ERROR'])


    # Convert to number - Remove '+' and ',' from 'PCT_ESTIMATE' and 'PRF_ESTIMATE', then convert to float
    for col in ['PCT_ESTIMATE', 'PRF_ESTIMATE']:
        data[col] = data[col].str.replace(',', '').str.replace('+', '').astype(float)
    
    return data


In [55]:
AL_econ_df = clean_up_data_econ(AL_econ_df)

In [56]:
format_districts(AL_econ_df)

,GEOID,District,PROFLN,TITLE,PRF_ESTIMATE,PCT_ESTIMATE,Formatted_District
3,620L800US01001,"State House District 1 (2022), Alabama",0.8-1,EMPLOYMENT STATUS - Population 16 years and over,39724.0,39724.0,AL-House-01
4,620L800US01001,"State House District 1 (2022), Alabama",0.8-1-2,EMPLOYMENT STATUS - Population 16 years and over - In labor force,23204.0,58.4,AL-House-01
5,620L800US01001,"State House District 1 (2022), Alabama",0.8-1-3,EMPLOYMENT STATUS - Population 16 years and over - Civilian labor force,23157.0,58.3,AL-House-01
6,620L800US01001,"State House District 1 (2022), Alabama",0.8-1-4,EMPLOYMENT STATUS - Population 16 years and over - Employed,22215.0,55.9,AL-House-01
7,620L800US01001,"State House District 1 (2022), Alabama",0.8-1-5,EMPLOYMENT STATUS - Population 16 years and over - Unemployed,942.0,2.4,AL-House-01
...,...,...,...,...,...,...,...
17635,620L800US01105,"State House District 105 (2022), Alabama",118.5-128-133,INCOME BELOW POVERTY LEVEL - All people - 18 years and over,13.2,13.2,AL-House-105
17636,620L800US01105,"State House District 105 (2022), Alabama",118.5-128-134,INCOME BELOW POVERTY LEVEL - All people - 18 to 64 years,14.3,14.3,AL-House-105
17637,620L800US01105,"State House District 105 (2022), Alabama",118.5-128-135,INCOME BELOW POVERTY LEVEL - All people - 65 years and over,9.6,9.6,AL-House-105
17638,620L800US01105,"State House District 105 (2022), Alabama",118.5-128-136,INCOME BELOW POVERTY LEVEL - All people - People in families,12.4,12.4,AL-House-105


In [57]:
AL_econ_prf, AL_econ_pct = reorient_dfs(AL_econ_df)
AL_econ_prf

,Formatted_District,GEOID,CLASS OF WORKER - Government workers,CLASS OF WORKER - Private wage and salary workers,CLASS OF WORKER - Self-employed in own not incorporated business workers,CLASS OF WORKER - Unpaid family workers,"COMMUTING TO WORK - Car, truck, or van -- carpooled","COMMUTING TO WORK - Car, truck, or van -- drove alone",COMMUTING TO WORK - Mean travel time to work (minutes),COMMUTING TO WORK - Other means,...,INDUSTRY - Public administration,INDUSTRY - Retail trade,"INDUSTRY - Transportation and warehousing, and utilities",INDUSTRY - Wholesale trade,"OCCUPATION - Management, business, science, and arts occupations","OCCUPATION - Natural resources, construction, and maintenance occupations","OCCUPATION - Production, transportation, and material moving occupations",OCCUPATION - Sales and office occupations,OCCUPATION - Service occupations,Total households
0,AL-House-01,620L800US01001,3475.0,17427.0,1284.0,29.0,1642.0,19178.0,23.8,334.0,...,992.0,3461.0,1089.0,723.0,6888.0,2129.0,4034.0,5379.0,3785.0,19955.0
1,AL-House-02,620L800US01002,4036.0,17356.0,1422.0,59.0,2259.0,18636.0,31.1,117.0,...,1274.0,3425.0,1543.0,471.0,6708.0,2983.0,4059.0,5453.0,3670.0,19723.0
2,AL-House-03,620L800US01003,2630.0,17306.0,908.0,78.0,1503.0,17626.0,19.2,160.0,...,775.0,2674.0,1228.0,422.0,6523.0,1994.0,4227.0,4194.0,3984.0,20269.0
3,AL-House-04,620L800US01004,3781.0,17753.0,980.0,40.0,949.0,18901.0,21.9,26.0,...,1586.0,2148.0,1026.0,514.0,11543.0,1186.0,2996.0,4317.0,2512.0,18659.0
4,AL-House-05,620L800US01005,3818.0,17813.0,1146.0,19.0,1543.0,19688.0,26.6,328.0,...,1663.0,2708.0,880.0,385.0,8007.0,2350.0,4295.0,4661.0,3483.0,19081.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,AL-House-95,620L800US01095,2597.0,18239.0,2374.0,4.0,1648.0,18827.0,22.5,297.0,...,1003.0,3682.0,821.0,483.0,7367.0,1901.0,2459.0,6672.0,4815.0,21587.0
101,AL-House-96,620L800US01096,3531.0,17632.0,1041.0,14.0,1363.0,18872.0,25.8,84.0,...,1347.0,2671.0,1081.0,740.0,9863.0,1690.0,2634.0,4864.0,3167.0,17302.0
102,AL-House-97,620L800US01097,2334.0,14888.0,808.0,21.0,1147.0,14921.0,21.6,354.0,...,817.0,1826.0,1265.0,294.0,6185.0,1650.0,3025.0,3126.0,4065.0,18377.0
103,AL-House-98,620L800US01098,2848.0,14043.0,396.0,0.0,880.0,15197.0,24.7,157.0,...,596.0,1616.0,1207.0,245.0,5536.0,1606.0,3009.0,3669.0,3467.0,17643.0


In [58]:
## PROCESS ALL 49 STATES HOUSING FUNCTION
def process_econ(file_state_list):
    prf_dfs = []
    pct_dfs = []
    
    for file_state in file_state_list:
        csv, state = file_state

        data = load_and_rename_data(csv)
        data = update_titles_econ(data)
        data = clean_up_data_econ(data)
        data = format_districts(data)

        prf_estimate_df, pct_estimate_df = reorient_dfs(data)

        prf_dfs.append(prf_estimate_df)
        pct_dfs.append(pct_estimate_df)
    
    df_prfecon_all = pd.concat(prf_dfs, ignore_index=True)
    df_pctecon_all = pd.concat(pct_dfs, ignore_index=True)
    
    return df_prfecon_all, df_pctecon_all


### Again, a few districts had a distinct error, so I redefined the above function to identify and assess how to handle them

In [60]:
def process_econ(file_state_list):
    prf_dfs = []
    pct_dfs = []
    error_files = []

    for file_state in file_state_list:
        csv, state = file_state
        try:
            data = load_and_rename_data(csv)
            data = update_titles_econ(data)
            data = clean_up_data_econ(data)
            data = format_districts(data)

            prf_estimate_df, pct_estimate_df = reorient_dfs(data)

            prf_dfs.append(prf_estimate_df)
            pct_dfs.append(pct_estimate_df)

        except ValueError as e:
            print(f"Error processing file: {csv}")
            error_files.append((csv, str(e)))
            # Optionally, print or log the rows causing the error
            problematic_rows = data[(data['PCT_ESTIMATE'].str.contains('-', na=False)) |
                                    (data['PRF_ESTIMATE'].str.contains('-', na=False))]
            print(problematic_rows)

    df_prfecon_all = pd.concat(prf_dfs, ignore_index=True) if prf_dfs else pd.DataFrame()
    df_pctecon_all = pd.concat(pct_dfs, ignore_index=True) if pct_dfs else pd.DataFrame()

    return df_prfecon_all, df_pctecon_all, error_files

# Usage
file_state_list = [
 ('data/lower/ALLSLDL_DP03_AK.csv', 'AK'),('data/lower/ALLSLDL_DP03_AL.csv', 'AL'),('data/lower/ALLSLDL_DP03_AR.csv', 'AR'),
 ('data/lower/ALLSLDL_DP03_AZ.csv', 'AZ'),('data/lower/ALLSLDL_DP03_CA.csv', 'CA'),('data/lower/ALLSLDL_DP03_CO.csv', 'CO'),
 ('data/lower/ALLSLDL_DP03_CT.csv', 'CT'),('data/lower/ALLSLDL_DP03_DE.csv', 'DE'),('data/lower/ALLSLDL_DP03_FL.csv', 'FL'),
 ('data/lower/ALLSLDL_DP03_GA.csv', 'GA'),('data/lower/ALLSLDL_DP03_HI.csv', 'HI'),('data/lower/ALLSLDL_DP03_IA.csv', 'IA'),
 ('data/lower/ALLSLDL_DP03_ID.csv', 'ID'),('data/lower/ALLSLDL_DP03_IL.csv', 'IL'),('data/lower/ALLSLDL_DP03_IN.csv', 'IN'),
 ('data/lower/ALLSLDL_DP03_KS.csv', 'KS'),('data/lower/ALLSLDL_DP03_KY.csv', 'KY'),('data/lower/ALLSLDL_DP03_LA.csv', 'LA'),
 ('data/lower/ALLSLDL_DP03_MA.csv', 'MA'),('data/lower/ALLSLDL_DP03_MD.csv', 'MD'),('data/lower/ALLSLDL_DP03_ME.csv', 'ME'),
 ('data/lower/ALLSLDL_DP03_MI.csv', 'MI'),('data/lower/ALLSLDL_DP03_MN.csv', 'MN'),('data/lower/ALLSLDL_DP03_MO.csv', 'MO'),
 ('data/lower/ALLSLDL_DP03_MS.csv', 'MS'),('data/lower/ALLSLDL_DP03_MT.csv', 'MT'),('data/lower/ALLSLDL_DP03_NC.csv', 'NC'),
 ('data/lower/ALLSLDL_DP03_ND.csv', 'ND'),('data/lower/ALLSLDL_DP03_NH.csv', 'NH'),
 ('data/lower/ALLSLDL_DP03_NJ.csv', 'NJ'),('data/lower/ALLSLDL_DP03_NM.csv', 'NM'),('data/lower/ALLSLDL_DP03_NV.csv', 'NV'),
 ('data/lower/ALLSLDL_DP03_NY.csv', 'NY'),('data/lower/ALLSLDL_DP03_OH.csv', 'OH'),('data/lower/ALLSLDL_DP03_OK.csv', 'OK'),
 ('data/lower/ALLSLDL_DP03_OR.csv', 'OR'),('data/lower/ALLSLDL_DP03_PA.csv', 'PA'),('data/lower/ALLSLDL_DP03_RI.csv', 'RI'),
 ('data/lower/ALLSLDL_DP03_SC.csv', 'SC'),('data/lower/ALLSLDL_DP03_SD.csv', 'SD'),('data/lower/ALLSLDL_DP03_TN.csv', 'TN'),
 ('data/lower/ALLSLDL_DP03_TX.csv', 'TX'),('data/lower/ALLSLDL_DP03_UT.csv', 'UT'),('data/lower/ALLSLDL_DP03_VA.csv', 'VA'),
 ('data/lower/ALLSLDL_DP03_VT.csv', 'VT'),('data/lower/ALLSLDL_DP03_WA.csv', 'WA'),('data/lower/ALLSLDL_DP03_WI.csv', 'WI'),
 ('data/lower/ALLSLDL_DP03_WV.csv', 'WV'),('data/lower/ALLSLDL_DP03_WY.csv', 'WY')
]

df_prfecon_all, df_pctecon_all, error_files = process_econ(file_state_list)

if error_files:
    print("Files with errors:", error_files)


In [61]:
df_pctecon_all

,Formatted_District,GEOID,CLASS OF WORKER - Government workers,CLASS OF WORKER - Private wage and salary workers,CLASS OF WORKER - Self-employed in own not incorporated business workers,CLASS OF WORKER - Unpaid family workers,"COMMUTING TO WORK - Car, truck, or van -- carpooled","COMMUTING TO WORK - Car, truck, or van -- drove alone",COMMUTING TO WORK - Mean travel time to work (minutes),COMMUTING TO WORK - Other means,...,INDUSTRY - Public administration,INDUSTRY - Retail trade,"INDUSTRY - Transportation and warehousing, and utilities",INDUSTRY - Wholesale trade,"OCCUPATION - Management, business, science, and arts occupations","OCCUPATION - Natural resources, construction, and maintenance occupations","OCCUPATION - Production, transportation, and material moving occupations",OCCUPATION - Sales and office occupations,OCCUPATION - Service occupations,Total households
0,AK-House-01,620L800US02001,29.9,61.7,8.0,0.3,16.1,66.2,13.5,3.2,...,12.0,10.6,10.5,1.6,34.2,12.0,13.2,20.5,20.1,6929.0
1,AK-House-02,620L800US02002,31.0,57.6,11.0,0.3,9.8,58.5,11.4,6.0,...,11.2,12.0,7.6,1.3,34.7,12.5,13.4,20.5,18.9,7170.0
2,AK-House-03,620L800US02003,39.5,49.6,10.7,0.1,10.0,64.7,14.8,3.3,...,21.3,6.7,8.4,1.1,42.7,13.5,10.5,19.4,13.8,6436.0
3,AK-House-04,620L800US02004,41.0,50.7,8.0,0.3,14.3,62.3,14.9,2.3,...,25.5,7.1,7.3,0.7,44.7,10.4,9.2,19.3,16.4,7866.0
4,AK-House-05,620L800US02005,26.2,65.0,8.5,0.2,16.3,59.4,10.2,9.2,...,12.2,9.0,6.7,2.1,30.2,15.5,20.6,17.8,15.9,6243.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4829,WY-House-58,620L800US56058,10.5,84.2,5.3,0.0,10.7,80.5,19.7,0.4,...,2.8,10.6,9.1,1.4,29.3,19.9,18.8,14.0,17.9,3312.0
4830,WY-House-59,620L800US56059,14.9,79.7,5.1,0.4,10.3,82.5,17.3,2.2,...,3.9,15.5,5.0,2.8,22.9,15.6,14.5,21.9,25.1,4049.0
4831,WY-House-60,620L800US56060,18.2,79.1,2.7,0.0,13.5,76.2,23.3,0.2,...,4.9,7.2,10.9,0.6,33.4,15.9,17.3,19.0,14.4,3054.0
4832,WY-House-61,620L800US56061,17.6,78.8,2.5,1.1,7.3,78.3,18.0,3.4,...,9.3,32.5,6.0,0.6,25.6,15.6,16.7,22.9,19.3,4158.0


# Merging all Characteristic Profile Datasets

In [62]:
#merge all the PERCENT dataframes

df_pct_merged = pd.merge(df_pctdemo_all, df_pcthousing_all, on=['Formatted_District', 'GEOID'])
df_pct_merged = pd.merge(df_pct_merged, df_pctsocial_all, on=['Formatted_District', 'GEOID'])
df_pct_merged = pd.merge(df_pct_merged, df_pctecon_all, on=['Formatted_District', 'GEOID'])
# check shape to ensure we maintained all 435 districts
print(df_pct_merged.head())
print(df_pct_merged.shape)

  Formatted_District           GEOID  \
0        AK-House-01  620L800US02001   
1        AK-House-02  620L800US02002   
2        AK-House-03  620L800US02003   
3        AK-House-04  620L800US02004   
4        AK-House-05  620L800US02005   

   CITIZEN, VOTING AGE POPULATION - Citizen, 18 and over population  \
0                                                           13500.0   
1                                                           13926.0   
2                                                           12818.0   
3                                                           14859.0   
4                                                           12765.0   

   CITIZEN, VOTING AGE POPULATION - Female  \
0                                     47.1   
1                                     46.2   
2                                     50.7   
3                                     47.6   
4                                     44.9   

   CITIZEN, VOTING AGE POPULATION - Male  HISPANIC OR L

In [63]:
#merge all the RAW NUMBER (PRF) dataframes

df_prf_merged = pd.merge(df_prfdemo_all, df_prfhousing_all, on=['Formatted_District', 'GEOID'])
df_prf_merged = pd.merge(df_prf_merged, df_prfsocial_all, on=['Formatted_District', 'GEOID'])
df_prf_merged = pd.merge(df_prf_merged, df_prfecon_all, on=['Formatted_District', 'GEOID'])
# check shape to ensure we maintained all 435 districts
print(df_prf_merged.head())
print(df_prf_merged.shape)

  Formatted_District           GEOID  \
0        AK-House-01  620L800US02001   
1        AK-House-02  620L800US02002   
2        AK-House-03  620L800US02003   
3        AK-House-04  620L800US02004   
4        AK-House-05  620L800US02005   

   CITIZEN, VOTING AGE POPULATION - Citizen, 18 and over population  \
0                                                           13500.0   
1                                                           13926.0   
2                                                           12818.0   
3                                                           14859.0   
4                                                           12765.0   

   CITIZEN, VOTING AGE POPULATION - Female  \
0                                   6362.0   
1                                   6427.0   
2                                   6502.0   
3                                   7075.0   
4                                   5733.0   

   CITIZEN, VOTING AGE POPULATION - Male  HISPANIC OR L

# Tidying Up

In [64]:
# duplicated columns between datsets
columns_to_delete = [
    'HOUSING OCCUPANCY - Total housing units',
    'PLACE OF BIRTH - Total population',
    'EMPLOYMENT STATUS - Population 16 years and over',
    'HOUSING OCCUPANCY - Occupied housing units',
    'COMPUTERS AND INTERNET USE - Total households',
    'Population in households - Householder',
    'Population born outside the United States - Native',
    'Total Civilian Noninstitutionalized Population'
]


# deleting the columns
df_pct_merged = df_pct_merged.drop(columns=columns_to_delete)
df_prf_merged = df_prf_merged.drop(columns=columns_to_delete)


In [65]:
# Identify numeric columns
numeric_cols_prf = df_prf_merged.select_dtypes(include=np.number).columns.tolist()
numeric_cols_pct = df_pct_merged.select_dtypes(include=np.number).columns.tolist()

# Replace NaNs in numeric columns with their median
df_prf_merged.loc[:, numeric_cols_prf] = df_prf_merged.loc[:, numeric_cols_prf].fillna(df_prf_merged[numeric_cols_prf].median())
df_pct_merged.loc[:, numeric_cols_pct] = df_pct_merged.loc[:, numeric_cols_pct].fillna(df_pct_merged[numeric_cols_pct].median())
    

In [66]:


# Find columns with more than 100 NaNs
cols_to_drop_prf = df_prf_merged.columns[df_prf_merged.isnull().sum() > 100].tolist()
cols_to_drop_pct = df_pct_merged.columns[df_pct_merged.isnull().sum() > 100].tolist()

# # Exclude 'Other Votes' from the list
cols_to_drop_prf = [col for col in cols_to_drop_prf]
cols_to_drop_pct = [col for col in cols_to_drop_pct]

# Drop these columns
df_prf_merged.drop(columns=cols_to_drop_prf, inplace=True)
df_pct_merged.drop(columns=cols_to_drop_pct, inplace=True)

# For 'RACE - ...' and 'LANGUAGE SPOKEN AT HOME...' categories, replace NaNs with 0
race_language_cols_prf = [col for col in df_prf_merged if 'RACE -' in col or 'LANGUAGE SPOKEN AT HOME' in col]
race_language_cols_pct = [col for col in df_pct_merged if 'RACE -' in col or 'LANGUAGE SPOKEN AT HOME' in col]

df_prf_merged[race_language_cols_prf] = df_prf_merged[race_language_cols_prf].fillna(0)
df_pct_merged[race_language_cols_pct] = df_pct_merged[race_language_cols_pct].fillna(0)

# For the remaining numeric columns, replace NaNs with the column's median
df_prf_merged.loc[:, numeric_cols_prf] = df_prf_merged.loc[:, numeric_cols_prf].fillna(df_prf_merged[numeric_cols_prf].median())
df_pct_merged.loc[:, numeric_cols_pct] = df_pct_merged.loc[:, numeric_cols_pct].fillna(df_pct_merged[numeric_cols_pct].median())



In [67]:
df_pct_merged.shape

(4834, 453)

# Train PVI Model on Congressional Districts where we know the PVI scores

In [71]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [72]:
# Load datasets
pct_df = pd.read_csv('data/Full_Dataset_PCT.csv')  
prf_df = pd.read_csv('data/Full_Dataset_PRF.csv')  

# Define target variable
y = pct_df['PVI_2023']

# Define feature variables
X = pct_df.iloc[:, 3:454]  # first two columns are titles of districts, last columns are potential targets

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [73]:
# Retrain the model with the best metrics on our data
pipeline = Pipeline([
    ('scaler', MinMaxScaler()),
    ('regressor', Ridge(alpha=1))
])

# Train the model on the training data
best_model_pvi = pipeline.fit(X_train, y_train)

# Make predictions on the test data
y_pred = best_model_pvi.predict(X_test)

# Calculate the R² score, RMSE, and MAE to confirm it's properly fit
r2_test = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)

# Display the metrics
print(f"R² on Test Data: {r2_test:.4f}")
print(f"RMSE on Test Data: {rmse:.4f}")
print(f"MAE on Test Data: {mae:.4f}")


R² on Test Data: 0.9122
RMSE on Test Data: 4.3654
MAE on Test Data: 3.2661


# Check to see that the same features are present to match the trained model

In [74]:
# Define the columns from USCD that are used as features in the model (excluding target and other non-relevant columns)
feature_columns = [col for col in pct_df.columns if col not in {'Unnamed: 0', 'Representative', 'PVI_2023', 'Rep', 'Status', 'Dem Votes', 'Rep Votes', 'Other Votes', 'Dem %', 'Rep %', 'Other %', '2022 Margin', 'Swing from 2020 presidential', 'Percent of 2020 turnout', '2022 State PVI', '2020 Biden %', '2020 Trump %', '2016 Clinton %', '2016 Trump %', 'National 2020 Pres Popular %', 'National 2022 House Popular %', '2020 State Diff', '2016 State Diff'}]

# Filter the state legislative districts dataframe to only include the feature columns
df_state_leg_filtered = df_pct_merged[feature_columns]

# Check if there are any missing columns in the state legislative districts dataframe
missing_columns = [col for col in feature_columns if col not in df_pct_merged.columns]
if missing_columns:
    print("Missing columns in state legislative districts dataset:", missing_columns)
else:
    print("All feature columns are present.")


All feature columns are present.


# Now let's use the trained model to assign predicted PVI scores for each State Legislative District's Lower Chamber

In [75]:
# Assume that the first two columns to be excluded are 'Formatted_District' and 'GEOID',
# and you need to take a specific range of columns based on how the features were arranged in the original dataset

# Using .iloc to select columns by index
X_state_leg = df_pct_merged.iloc[:, 2:453]  # Adjust indices to match feature columns; starting from index 2 to exclude the first two

# Use the trained model to predict PVI
df_pct_merged['PVI'] = best_model_pvi.predict(X_state_leg)

# Output the first few rows to verify the new PVI column
print(df_pct_merged[['Formatted_District', 'GEOID', 'PVI']].head())

# If you need to save the updated DataFrame:
df_pct_merged.to_csv('State_Legislative_Districts_with_PVI.csv', index=False)
print("PVI scores predicted and saved successfully.")


  Formatted_District           GEOID        PVI
0        AK-House-01  620L800US02001 -13.791318
1        AK-House-02  620L800US02002 -18.109003
2        AK-House-03  620L800US02003  -2.644022
3        AK-House-04  620L800US02004   3.052562
4        AK-House-05  620L800US02005  -7.662132
PVI scores predicted and saved successfully.


In [76]:
df_pct_merged.to_csv('data/ACS_ALLSLDL_PVI.csv')

In [77]:
df_pct_merged.shape

(4834, 454)

## Assess predicted PVI for all Upper House State Seats in USA

In [127]:
sldl_pvi_df = df_pct_merged[["GEOID","PVI"]]

In [128]:
#FIPS codes used in the GEOID to represent values for States
# Will use to make a column for states in my df

fips_to_state = {
    '01': 'AL', '02': 'AK', '04': 'AZ', '05': 'AR', '06': 'CA',
    '08': 'CO', '09': 'CT', '10': 'DE', '11': 'DC', '12': 'FL',
    '13': 'GA', '15': 'HI', '16': 'ID', '17': 'IL', '18': 'IN',
    '19': 'IA', '20': 'KS', '21': 'KY', '22': 'LA', '23': 'ME',
    '24': 'MD', '25': 'MA', '26': 'MI', '27': 'MN', '28': 'MS',
    '29': 'MO', '30': 'MT', '31': 'NE', '32': 'NV', '33': 'NH',
    '34': 'NJ', '35': 'NM', '36': 'NY', '37': 'NC', '38': 'ND',
    '39': 'OH', '40': 'OK', '41': 'OR', '42': 'PA', '44': 'RI',
    '45': 'SC', '46': 'SD', '47': 'TN', '48': 'TX', '49': 'UT',
    '50': 'VT', '51': 'VA', '53': 'WA', '54': 'WV', '55': 'WI',
    '56': 'WY', '72': 'PR'  # Including Puerto Rico
}


In [129]:
# Extract state FIPS codes as a new column
sldl_pvi_df = sldl_pvi_df.copy()
sldl_pvi_df['state_fips'] = sldl_pvi_df['GEOID'].apply(lambda x: x.split('US')[-1][:2])

# Map FIPS codes to state abbreviations
sldl_pvi_df['state'] = sldl_pvi_df['state_fips'].map(fips_to_state)


In [130]:
actual_sen_seats_SLDL = {'Democratic': 2434, 'Republican': 2941}

In [134]:
predicted_dems_SLDL = sum(sldl_pvi_df['PVI']>0)
predicted_reps_SLDL = sum(sldl_pvi_df['PVI']<0)
tossup_districts = sum((sldl_pvi_df['PVI'] < 4.35) & (df_pct_merged['PVI'] > -4.35))


In [135]:
print(predicted_dems_SLDL)
print(predicted_reps_SLDL)
print(tossup_districts)

2164
2670
224


In [133]:
sldl_pvi_df['PVI'] = sldl_pvi_df['PVI']+6.25

# Assess predicted PVI performance on a test state's lower chamber

In [79]:
nc_reps = pd.read_csv('NCGA_House.csv')

In [80]:
nc_reps['District'].dtype

dtype('int64')

In [81]:
df_pct_merged[['Formatted_District', 'GEOID', 'PVI']]

# Prepare PVI data for NC
nc_filtered = df_pct_merged[df_pct_merged['Formatted_District'].str.startswith('NC')]
nc_pvi = nc_filtered[['GEOID', 'Formatted_District', 'PVI']].rename(columns={
    'PVI': 'pred_PVI',  # Ensure this is within 10 characters
    'Formatted_District': 'district'  # Ensure this is within 10 characters
})

In [82]:
nc_pvi['District_Number'] = nc_pvi['district'].str.split('House-').str[1].astype(int)

In [83]:
nc_pvi.columns

Index(['GEOID', 'district', 'pred_PVI', 'District_Number'], dtype='object')

In [84]:
nc_reps_pvi_df = nc_pvi.merge(nc_reps, left_on='District_Number', right_on='District', how='left')

In [85]:
nc_reps_pvi_df

,GEOID,district,pred_PVI,District_Number,Party,District,Member,Counties Represented
0,620L800US37001,NC-House-01,-19.653329,1,R,1,Edward C. Goodwin,"Chowan, Currituck, Dare, Perquimans, Tyrrell, Washington"
1,620L800US37002,NC-House-02,-6.925117,2,D,2,B. Ray Jeffers,"Durham, Person"
2,620L800US37003,NC-House-03,-13.867474,3,R,3,Steve Tyson,Craven
3,620L800US37004,NC-House-04,-11.229044,4,R,4,Jimmy Dixon,"Duplin, Wayne"
4,620L800US37005,NC-House-05,-9.385555,5,R,5,Bill Ward,"Camden, Gates, Hertford, Pasquotank"
...,...,...,...,...,...,...,...,...
115,620L800US37095,NC-House-95,-12.840778,95,R,95,Grey Mills,Iredell
116,620L800US37096,NC-House-96,-15.418916,96,R,96,Jay Adams,Catawba
117,620L800US37097,NC-House-97,-31.539435,97,R,97,Jason Saine,Lincoln
118,620L800US37098,NC-House-98,-5.555210,98,R,98,"John R. Bradford, III",Mecklenburg


In [87]:
sorted_nc_pvi = nc_reps_pvi_df.sort_values(by='pred_PVI', ascending=True)
sorted_nc_pvi[["district","pred_PVI","Party", "Member", "Counties Represented"]]

,district,pred_PVI,Party,Member,Counties Represented
114,NC-House-94,-34.515696,R,Jeffrey Elmore,"Alexander, Wilkes"
117,NC-House-97,-31.539435,R,Jason Saine,Lincoln
107,NC-House-87,-31.444805,R,Destin Hall,"Caldwell, Watauga"
109,NC-House-89,-30.111367,R,Mitchell S. Setzer,"Catawba, Iredell"
32,NC-House-120,-29.056698,R,Karl E. Gillespie,"Cherokee, Clay, Graham, Macon"
...,...,...,...,...,...
108,NC-House-88,24.930313,D,Mary Belk,Mecklenburg
58,NC-House-38,28.121603,D,Abe Jones,Wake
119,NC-House-99,28.405656,D,Nasif Majeed,Mecklenburg
12,NC-House-102,29.757190,D,Becky Carney,Mecklenburg


In [88]:
pd.set_option('display.max_rows', None)  # This will allow all rows to be displayed.
pd.set_option('display.max_columns', None)  # This will allow all columns to be displayed.
pd.set_option('display.width', None)  # This will help to ensure that the display is not truncated horizontally.
pd.set_option('display.max_colwidth', None)  # This will ensure that the content of each column is fully displayed.

sorted_nc_pvi[["district","pred_PVI","Party", "Member", "Counties Represented"]]


,district,pred_PVI,Party,Member,Counties Represented
114,NC-House-94,-34.515696,R,Jeffrey Elmore,"Alexander, Wilkes"
117,NC-House-97,-31.539435,R,Jason Saine,Lincoln
107,NC-House-87,-31.444805,R,Destin Hall,"Caldwell, Watauga"
109,NC-House-89,-30.111367,R,Mitchell S. Setzer,"Catawba, Iredell"
32,NC-House-120,-29.056698,R,Karl E. Gillespie,"Cherokee, Clay, Graham, Macon"
110,NC-House-90,-28.696081,R,Sarah Stevens,"Surry, Wilkes"
87,NC-House-67,-28.448736,R,Wayne Sasser,"Montgomery, Stanly"
97,NC-House-77,-28.282246,R,Julia C. Howard,"Davie, Rowan, Yadkin"
100,NC-House-80,-27.052053,R,Sam Watford,Davidson
103,NC-House-83,-26.640137,R,Kevin Crutchfield,"Cabarrus, Rowan"


In [ ]:
south carolina house 82

In [140]:
sldl_pvi_df

,GEOID,PVI,state_fips,state
0,620L800US02001,-7.541318,02,AK
1,620L800US02002,-11.859003,02,AK
2,620L800US02003,3.605978,02,AK
3,620L800US02004,9.302562,02,AK
4,620L800US02005,-1.412132,02,AK
5,620L800US02006,-3.841296,02,AK
6,620L800US02007,-22.512458,02,AK
7,620L800US02008,-20.667253,02,AK
8,620L800US02009,5.491334,02,AK
9,620L800US02010,6.540365,02,AK
